In [3]:
%matplotlib inline
import sys
import datetime as dt
import numpy as np
import numpy.matlib as matlib
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import select, and_

import config
from model import Market

from PyFin.api import *
from PyFin.api.Analysis import DELTA
from PyFin.Math.Accumulators import Log
from PyFin.Math.Accumulators import MovingRank
from PyFin.Math.Accumulators.IAccumulators import Pow
from PyFin.Math.Accumulators.StatefulAccumulators import MovingCorrelation
from PyFin.Analysis.CrossSectionValueHolders import CSRankedSecurityValueHolder
from PyFin.Analysis.CrossSectionValueHolders import CSPercentileSecurityValueHolder
from PyFin.Analysis.SecurityValueHolders import SecurityIIFValueHolder
from PyFin.Analysis.SecurityValueHolders import SecurityDeltaValueHolder
from PyFin.Analysis.SecurityValueHolders import SecurityShiftedValueHolder
from PyFin.Analysis.SecurityValueHolders import SecurityLatestValueHolder
from PyFin.Analysis.TechnicalAnalysis.StatelessTechnicalAnalysers import SecurityMinimumValueHolder
from PyFin.Analysis.TechnicalAnalysis.StatelessTechnicalAnalysers import SecurityMaximumValueHolder
from PyFin.Analysis.TechnicalAnalysis.StatelessTechnicalAnalysers import SecurityExpValueHolder
from PyFin.Analysis.TechnicalAnalysis.StatelessTechnicalAnalysers import SecuritySqrtValueHolder
from PyFin.Analysis.TechnicalAnalysis.StatelessTechnicalAnalysers import SecuritySignValueHolder
from PyFin.Analysis.TechnicalAnalysis.StatelessTechnicalAnalysers import SecurityLogValueHolder
from PyFin.Analysis.TechnicalAnalysis.StatelessTechnicalAnalysers import SecuritySignValueHolder
from PyFin.Analysis.TechnicalAnalysis.StatelessTechnicalAnalysers import SecurityPowValueHolder
from PyFin.Analysis.TechnicalAnalysis.StatelessTechnicalAnalysers import SecurityXAverageValueHolder
from PyFin.Analysis.TechnicalAnalysis.StatefulTechnicalAnalysers import SecurityMovingMax
from PyFin.Analysis.TechnicalAnalysis import SecurityAbsValueHolder
from PyFin.Analysis.TechnicalAnalysis import SecurityMovingAverage
from PyFin.Analysis.TechnicalAnalysis import SecurityMovingCorrelation
from PyFin.Analysis.TechnicalAnalysis import SecurityMovingSum
from PyFin.Analysis.TechnicalAnalysis import SecurityMovingStandardDeviation
from PyFin.Analysis.TechnicalAnalysis import SecurityMovingRank
from PyFin.Analysis.TechnicalAnalysis import SecurityMovingMin

In [4]:
engine = create_engine(config.DX_PUB_URL)
begin_date = '2018-06-01'
end_date = '2018-12-28'

In [5]:
query = select([Market]).where(
            and_(Market.trade_date >= begin_date, Market.trade_date <= end_date, ))
mkt_df = pd.read_sql(query, engine)
mkt_df.rename(columns={'preClosePrice':'pre_close','openPrice':'openPrice',
                      'highestPrice':'highestPrice','lowestPrice':'lowestPrice',
                      'closePrice':'closePrice','turnoverVol':'turnoverVol',
                      'turnoverValue':'turnoverValue','accumAdjFactor':'accum_adj',
                       'vwap':'vwap'}, inplace=True)
mkt_df = mkt_df[[('000000' + str(code))[-6:][0] in '036' for code in mkt_df['code']]]
trade_date_list = list(set(mkt_df.trade_date))
trade_date_list.sort(reverse=True)
mkt_df = mkt_df.set_index('trade_date')
mkt_df = mkt_df[mkt_df['turnoverVol'] > 0]
#计算收益率

In [6]:
df = mkt_df.loc['2018-12-28']
df.head(5)

,code,secShortName,exchangeCD,pre_close,actPreClosePrice,openPrice,highestPrice,lowestPrice,closePrice,turnoverVol,...,turnoverRate,accum_adj,negMarketValue,marketValue,chgPct,PE,PE1,PB,isOpen,vwap
trade_date,,,,,,,,,,,,,,,,,,,,,
2018-12-28,1,平安银行,XSHE,9.28,9.28,9.31,9.46,9.31,9.38,57660400,...,0.0034,1.0,1.610569e+11,1.610585e+11,0.0108,6.5760,5.9051,0.6847,1,9.392
2018-12-28,2,万科A,XSHE,23.44,23.44,23.67,24.24,23.67,23.82,32281093,...,0.0033,1.0,2.314153e+11,2.629526e+11,0.0162,8.4973,14.1023,1.8782,1,23.937
2018-12-28,4,国农科技,XSHE,16.03,16.03,15.73,16.26,15.73,16.03,167100,...,0.0020,1.0,1.330702e+09,1.346147e+09,0.0000,117.9875,-389.5476,10.5936,1,16.071
2018-12-28,5,世纪星源,XSHE,2.66,2.66,2.66,2.69,2.65,2.68,3717158,...,0.0039,1.0,2.549276e+09,2.836879e+09,0.0075,74.5494,325.7310,2.1345,1,2.669
2018-12-28,6,深振业A,XSHE,5.16,5.16,5.22,5.23,5.15,5.18,5026323,...,0.0037,1.0,6.984235e+09,6.992975e+09,0.0039,7.2112,10.4219,1.2002,1,5.190


In [7]:
start_pos = '2018-10-01'
end_pos = end_date

### alpha191-1
$(-1*CORR(RANK(DELTA(LOG(VOLUME),1)),RANK(((CLOSE-OPEN)/OPEN)),6)$

In [6]:
%%time

#-1*CORR(expression1,RANK(expression2),6)

#RANK(DELTA(LOG(VOLUME),1))
expression1 = CSPercentileSecurityValueHolder(
    SecurityDeltaValueHolder(1, SecurityLogValueHolder(SecurityLatestValueHolder('turnoverVol'))))

#(CLOSE-OPEN)/OPEN)
expression2 = CSPercentileSecurityValueHolder((SecurityLatestValueHolder('closePrice') - \
                SecurityLatestValueHolder('openPrice')) / SecurityLatestValueHolder('openPrice'))


#expression3 = -1*CORR(expression1,RANK(expression2),6)
expression3 = -1 * SecurityMovingCorrelation(6, expression1, CSRankedSecurityValueHolder(expression2))
name = 'alpha1'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = expression3.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).loc[end_pos].head(5)

CPU times: user 1.58 s, sys: 0 ns, total: 1.58 s
Wall time: 1.58 s


,alpha1,code
trade_date,,
2018-12-28,0.284335,603999
2018-12-28,0.620483,2729
2018-12-28,0.424145,2740
2018-12-28,0.105340,2739
2018-12-28,-0.072108,2738


### alpha 191 -2 
$-1*delta(((close-low)-(high-close))/(high-low),1)$

In [7]:
%%time

#-1*delta(expression1/expression2,1)

#((close-low)-(high-close))/(high-low)
expression1 = SecurityLatestValueHolder('closePrice'
                         ) * 2  - SecurityLatestValueHolder('lowestPrice'
                                                           ) - SecurityLatestValueHolder('highestPrice')
#(high-low)
expression2 = SecurityLatestValueHolder('highestPrice') - SecurityLatestValueHolder('lowestPrice')

# -1*delta(expression1/expression2,1)
alpha2 = SecurityDeltaValueHolder(1, expression1 / expression2) * -1

name='alpha2'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = alpha2.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).loc[end_pos].head(5)

/home/kerry/work/workenv/sentry/lib/python3.6/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app


CPU times: user 1.09 s, sys: 0 ns, total: 1.09 s
Wall time: 1.08 s


,alpha2,code
trade_date,,
2018-12-28,-0.446154,603999
2018-12-28,-0.581818,2729
2018-12-28,-0.528571,2740
2018-12-28,-1.632353,2739
2018-12-28,-1.521739,2738


## alpha191-3
$-1*SUM((CLOSE=DELAY(CLOSE,1)?0:CLOSE-(CLOSE>DELAY(CLOSE,1)?MIN(LOW,DELAY(CLOSE,1)):MAX(HIGH,DELAY(CLOSE,1)))),6)$

In [8]:
%%time

#-1*SUM((CLOSE=DELAY(CLOSE,1)?0:CLOSE-(CLOSE>DELAY(CLOSE,1)?MIN(LOW,DELAY(CLOSE,1)):MAX(HIGH,DELAY(CLOSE,1)))),6)$



#CLOSE>DELAY(CLOSE,1)
expression1 = SecurityLatestValueHolder('closePrice') > SecurityShiftedValueHolder(1,
        SecurityLatestValueHolder('closePrice'))

#MIN(LOW,DELAY(CLOSE,1))
expression2 = SecurityMinimumValueHolder(SecurityLatestValueHolder('lowestPrice'),
                                         SecurityShiftedValueHolder(1, SecurityLatestValueHolder('closePrice')))

#MAX(HIGH,DELAY(CLOSE,1))
expression3 = SecurityMaximumValueHolder(SecurityLatestValueHolder('lowestPrice'),
                                         SecurityShiftedValueHolder(1, SecurityLatestValueHolder('closePrice')))

#CLOSE-(expression1?expression2:expression3)
expression4 = SecurityLatestValueHolder('closePrice') - SecurityIIFValueHolder(
            expression1,expression2,expression3) 


#(CLOSE=DELAY(CLOSE,1)?0:expression4)
expression5 = SecurityIIFValueHolder(SecurityLatestValueHolder('closePrice')==SecurityShiftedValueHolder(1, 
                                    SecurityLatestValueHolder('closePrice')),
                                    0, expression4)

#-1*SUM(expression5,6)

expression6 = SecurityMovingAverage(6, expression5)

name = 'alpha3'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = expression6.transform(df,name=name,category_field='code', dropna=False)
result.sort_values(by='trade_date', ascending=False).head(5)

/home/kerry/work/workenv/sentry/lib/python3.6/site-packages/ipykernel_launcher.py:33: RuntimeWarning: invalid value encountered in greater


CPU times: user 2.02 s, sys: 0 ns, total: 2.02 s
Wall time: 2.01 s


,alpha3,code
trade_date,,
2018-12-28,-0.046667,603999
2018-12-28,0.055000,2730
2018-12-28,0.033333,2741
2018-12-28,0.001667,2740
2018-12-28,-0.033333,2739


## alpha191-4
$(((SUM(CLOSE,8)/8)+STD(CLOSE,8))<(SUM(CLOSE,2)/2))?-1:(SUM(CLOSE,2)/2<(SUM(CLOSE,8)/8-STD(CLOSE,8))?1:(1<=(VOLUME/MEAN(VOLUME,20))?1:-1))$

In [9]:
%%time
#SUM(CLOSE,8)/8
expression1 = (SecurityMovingSum(8, 'closePrice') / 8)

# STD(CLOSE,8)
expression2 = SecurityMovingStandardDeviation(8, 'closePrice')

expression3 = expression1 + expression2

#(SUM(CLOSE,2)/2)
expression4 = (SecurityMovingSum(2, 'closePrice') / 2)


#VOLUME/MEAN(VOLUME,20)
expression6 = SecurityLatestValueHolder('turnoverVol') / SecurityMovingAverage(20, 'turnoverVol')

#(1<=(expression6)?1:-1)
expression7 = SecurityIIFValueHolder(1 < expression6, 1, -1)

# (expression1-expression2)?1:expression7
expression8 = SecurityIIFValueHolder(expression1-expression2,
                                    1, expression7)

#(expression3<expression4)?-1:(expression4<expression8)
expression9 = SecurityIIFValueHolder(expression3<expression4, -1, expression4<expression8)
name = 'alpha4'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = expression9.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

/home/kerry/work/workenv/sentry/lib/python3.6/site-packages/ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in less


ValueError: Does not understand character buffer dtype format string ('?')

## alpha191-5
$-1*TSMAX(CORR(TSRANK(VOLUME,5),TSRANK(HIGH,5),5),3)$

In [10]:
%%time
#-1*TSMAX(CORR(TSRANK(VOLUME,5),TSRANK(HIGH,5),5),3)

# TSRANK(VOLUME,5)
expression1 = SecurityMovingRank(5, 'turnoverVol')
# TSRANK(HIGH,5)
expression2 = SecurityMovingRank(5, 'highestPrice')

#CORR(expression1,expression2,5)
expression3 = SecurityMovingCorrelation(5, expression1 , expression2)

#TSMAX(expression3,3)
expression4 = SecurityMovingMax(3, expression3) * -1

name = 'alpha5'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = expression4.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

CPU times: user 2.09 s, sys: 0 ns, total: 2.09 s
Wall time: 2.09 s


,alpha5,code
trade_date,,
2018-12-28,0.091287,603999
2018-12-28,0.133631,2729
2018-12-28,-0.553509,2740
2018-12-28,-0.536020,2739
2018-12-28,-0.884652,2738


## alpha191-6
$-1*RANK(SIGN(DELTA(OPEN*x+HIGH*0.15,4)))$

In [11]:
%%time
#SIGN(DELTA(OPEN*0.85+HIGH*0.15,4))
expression1 = SecuritySignValueHolder(SecurityDeltaValueHolder(4,
        SecurityLatestValueHolder('openPrice') * 0.85 + SecurityLatestValueHolder('highestPrice') * 0.15))

#-1*RANK(expression2)
expression2 = CSPercentileSecurityValueHolder(expression1) * -1

name = 'alpha6'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = expression2.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

CPU times: user 1.03 s, sys: 0 ns, total: 1.03 s
Wall time: 1.03 s


,alpha6,code
trade_date,,
2018-12-28,-0.364187,603999
2018-12-28,-0.864751,2730
2018-12-28,-0.864751,2741
2018-12-28,-0.364187,2740
2018-12-28,-0.364187,2739


## alpha191-7
$(RANK(MAX(VWAP-CLOSE,3))+RANK(MIN(VWAP-CLOSE,3)))*RANK(DELTA(VOLUME,3))$

In [13]:
%%time
#RANK(MAX(VWAP-CLOSE,3))
expression1 = CSPercentileSecurityValueHolder(SecurityMaximumValueHolder(
            SecurityLatestValueHolder('vwap') - SecurityLatestValueHolder('closePrice'),3))
                                         
#RANK(MIN(VWAP-CLOSE,3))
expression2 = CSPercentileSecurityValueHolder(
            SecurityMinimumValueHolder(SecurityLatestValueHolder('vwap'
                                                                ) - SecurityLatestValueHolder('closePrice'),3))

#RANK(DELTA(VOLUME,3))
expression3 = SecurityDeltaValueHolder(3, SecurityLatestValueHolder('turnoverVol'))

#(expression1+expression2)*expression3
expression4 = (expression1 + expression2) * expression3
name = 'alpha7'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = expression4.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

CPU times: user 956 ms, sys: 0 ns, total: 956 ms
Wall time: 955 ms


,alpha7,code
trade_date,,
2018-12-28,-4.368892e+06,603999
2018-12-28,7.472300e+05,2730
2018-12-28,-9.921072e+04,2741
2018-12-28,-1.123124e+06,2740
2018-12-28,1.253072e+06,2739


## alpha191-8
$-1*RANK(DELTA((HIGH+LOW)/10+VWAP*0.8,4))$

In [14]:
%%time
#(HIGH+LOW)/10
exression1 = SecurityLatestValueHolder('highestPrice') + SecurityLatestValueHolder('lowestPrice') / 10

#VWAP*0.8
exression2 =  SecurityLatestValueHolder('vwap') * 0.8

#DELTA(exression1+exression2,4)
exression3 = SecurityDeltaValueHolder(4, exression1 + exression2)

#-1*RANK(exression3)
exression4 = CSPercentileSecurityValueHolder(exression3) * -1

name = 'alpha8'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = exression3.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

CPU times: user 860 ms, sys: 0 ns, total: 860 ms
Wall time: 860 ms


,alpha8,code
trade_date,,
2018-12-28,-0.4812,603999
2018-12-28,0.3222,2730
2018-12-28,-0.2356,2741
2018-12-28,-0.3788,2740
2018-12-28,-3.4316,2739


## alpha191-9
$SMA(((HIGH+LOW)/2-(DELAY(HIGH,1)+DELAY(LOW,1))/2)*(HIGH-LOW)/VOLUME,7,2)$
- 缺少 SMA公式

In [15]:
%%time
# (HIGH+LOW)/2
exression1 = (SecurityLatestValueHolder('highestPrice') + SecurityLatestValueHolder('lowestPrice')) / 2
# DELAY(HIGH,1)
exression2 = SecurityDeltaValueHolder(1, SecurityLatestValueHolder('highestPrice'))
# DELAY(LOW,1)
exression3 = SecurityDeltaValueHolder(1,SecurityLatestValueHolder('lowestPrice'))

# (exression1-(exression2 + exression3)/2)*(HIGH-LOW)/VOLUME
exression4 = (exression1 - (exression2 + exression3 ) / 2) * \
        (SecurityLatestValueHolder('highestPrice') - SecurityLatestValueHolder('lowestPrice'))\
        / SecurityLatestValueHolder('turnoverVol')

#SMA(exression4,7,2)
exression5 = SecurityMovingAverage(7, exression4)
name = 'alpha9'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = exression5.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

CPU times: user 1.7 s, sys: 0 ns, total: 1.7 s
Wall time: 1.7 s


,alpha9,code
trade_date,,
2018-12-28,2.487666e-07,603999
2018-12-28,3.631956e-05,2729
2018-12-28,8.581027e-07,2740
2018-12-28,1.747092e-06,2739
2018-12-28,9.751832e-06,2738


## alpha191-11
$SUM(((CLOSE-LOW)-(HIGH-CLOSE))/(HIGH-LOW)*VOLUME,6)$

In [16]:
%%time
#(CLOSE-LOW)-(HIGH-CLOSE)
exression1 = (SecurityLatestValueHolder('closePrice') - SecurityLatestValueHolder('lowestPrice')) - \
        (SecurityLatestValueHolder('highestPrice') - SecurityLatestValueHolder('closePrice'))

#expression1/(HIGH-LOW)*VOLUME
exression2 = exression1 / (SecurityLatestValueHolder('highestPrice') - SecurityLatestValueHolder('lowestPrice'))\
        * SecurityLatestValueHolder('turnoverVol')

#SUM(exression2,6)
exression3 = SecurityMovingSum(6, exression2)
name = 'alpha11'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = exression2.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

/home/kerry/work/workenv/sentry/lib/python3.6/site-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in true_divide
  del sys.path[0]


CPU times: user 668 ms, sys: 0 ns, total: 668 ms
Wall time: 660 ms


,alpha11,code
trade_date,,
2018-12-28,-7.463852e+05,603999
2018-12-28,-1.635997e+06,2730
2018-12-28,1.605240e+05,2741
2018-12-28,-3.008700e+05,2740
2018-12-28,8.876912e+06,2739


## alpha191-12
$RANK(OPEN-MA(VWAP,10))*RANK(ABS(CLOSE-VWAP))*(-1)$

In [17]:
%%time

#RANK(OPEN-MA(VWAP,10))
exression1 = CSPercentileSecurityValueHolder(
    SecurityLatestValueHolder('openPrice') - SecurityMovingAverage(10, 'vwap'))

#RANK(ABS(CLOSE-VWAP))
exression2 = CSPercentileSecurityValueHolder(SecurityAbsValueHolder(
        SecurityLatestValueHolder('closePrice') - SecurityLatestValueHolder('vwap')))

#exression1 * exression2 * -1
exression3 = exression1 * exression2 * -1
name = 'alpha12'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = exression2.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

CPU times: user 656 ms, sys: 0 ns, total: 656 ms
Wall time: 654 ms


,alpha12,code
trade_date,,
2018-12-28,0.329107,603999
2018-12-28,0.567202,2730
2018-12-28,0.755847,2741
2018-12-28,0.270921,2740
2018-12-28,0.933643,2739


## alpha191-13
$((HIGH*LOW)^{0.5})-VWAP$

In [18]:
#(HIGH*LOW)
expression1 = SecurityLatestValueHolder('highestPrice') * SecurityLatestValueHolder('lowestPrice')
#expression1^0.5
expression2 = SecurityPowValueHolder(expression1,0.5)

expression3 = expression2 - SecurityLatestValueHolder('vwap')
name = 'alpha13'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = expression3.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

,alpha13,code
trade_date,,
2018-12-28,-0.000257,603999
2018-12-28,0.039890,2730
2018-12-28,0.055340,2741
2018-12-28,0.003762,2740
2018-12-28,-0.138806,2739


## alpha191-14
$CLOSE-DELAY(CLOSE,5)$

In [19]:
alpha14 = SecurityLatestValueHolder('closePrice') - \
SecurityShiftedValueHolder(5, SecurityLatestValueHolder('closePrice'))
name = 'alpha14'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = alpha14.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

,alpha14,code
trade_date,,
2018-12-28,-0.30,603999
2018-12-28,-0.48,2729
2018-12-28,-0.21,2740
2018-12-28,-1.48,2739
2018-12-28,-0.53,2738


## alpha191-15
$OPEN/DELAY(CLOSE,1)-1$

In [20]:
alpha15 = SecurityLatestValueHolder('openPrice') /\
SecurityShiftedValueHolder(1,SecurityLatestValueHolder('closePrice')) - 1
name = 'alpha15'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = alpha15.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

,alpha15,code
trade_date,,
2018-12-28,0.010352,603999
2018-12-28,0.019099,2729
2018-12-28,0.007619,2740
2018-12-28,0.005174,2739
2018-12-28,0.012023,2738


## alpha191-16
$(-1*TSMAX(RANK(CORR(RANK(VOLUME),RANK(VWAP),5)),5))$

In [21]:
# (-1*TSMAX(RANK(CORR(exression1,RANK(VWAP),5)),5))

#RANK(VOLUME)
exression1 = CSPercentileSecurityValueHolder(SecurityLatestValueHolder('turnoverVol'))
#RANK(VWAP)
exression2 = CSPercentileSecurityValueHolder(SecurityLatestValueHolder('vwap'))
#CORR(expression1,expression2,5)
exression3 = SecurityMovingCorrelation(5, exression1,exression2)
#RANK(expression3)
exression4 = CSPercentileSecurityValueHolder(exression3)
#-1*TSMAX(RANK(expression3),5)
exression5 = SecurityMovingRank(5, exression4)
name = 'alpha16'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = exression5.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

,alpha16,code
trade_date,,
2018-12-28,3.0,603999
2018-12-28,0.0,2729
2018-12-28,4.0,2740
2018-12-28,0.0,2739
2018-12-28,1.0,2738


## alpha191-17
$RANK(VWAP-MAX(VWAP,15))^DELTA(CLOSE,5)$
- SecurityPowValueHolder 幂不支持公式

In [22]:
#DELTA(CLOSE,5)
expression1 = SecurityDeltaValueHolder(5, SecurityLatestValueHolder('closePrice'))
#VWAP-MAX(VWAP,15)
expression2 = SecurityLatestValueHolder('vwap')  - SecurityMovingMax(15, SecurityLatestValueHolder('vwap'))
#RANK(expression2)
expression3 = CSPercentileSecurityValueHolder(expression2)

#expression4 = SecurityPowValueHolder(expression3,expression1)
name = 'alpha17'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = expression3.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

,alpha17,code
trade_date,,
2018-12-28,0.547619,603999
2018-12-28,0.450831,2730
2018-12-28,0.162581,2741
2018-12-28,0.495069,2740
2018-12-28,0.069597,2739


## alpha191-18
$CLOSE/DELAY(CLOSE,5))$

In [23]:
alpha18 = SecurityLatestValueHolder('closePrice')/ SecurityShiftedValueHolder(5,
    SecurityLatestValueHolder('closePrice'))
name = 'alpha18'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = alpha15.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

,alpha18,code
trade_date,,
2018-12-28,0.010352,603999
2018-12-28,-0.001422,2730
2018-12-28,0.017699,2741
2018-12-28,0.007619,2740
2018-12-28,0.005174,2739


## alpha191-19
$(CLOSE<DELAY(CLOSE,5)?(CLOSE/DELAY(CLOSE,5)-1):(CLOSE=DELAY(CLOSE,5)?0:(1-DELAY(CLOSE,5)/CLOSE)))$

In [24]:
#DELAY(CLOSE,5)
exression1 = SecurityShiftedValueHolder(5, SecurityLatestValueHolder('closePrice'))

#CLOSE<expression1
exression2 = SecurityLatestValueHolder('closePrice') < exression1

#CLOSE/exression1-1
exression3 = SecurityLatestValueHolder('closePrice') / exression1- 1


#1-expression1/CLOSE
exression4 = 1 - expression1 / SecurityLatestValueHolder('closePrice')


#CLOSE=expression1?0:exression4
exression5 = SecurityIIFValueHolder(SecurityLatestValueHolder('closePrice')==exression1,
                                    0,exression4)
#exression2?exression3:exression5
exression6 = SecurityIIFValueHolder(exression2,exression3,exression5)
name = 'alpha19'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = exression6.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

/home/kerry/work/workenv/sentry/lib/python3.6/site-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in less


,alpha19,code
trade_date,,
2018-12-28,-0.058366,603999
2018-12-28,-0.022770,2729
2018-12-28,-0.038603,2740
2018-12-28,-0.063492,2739
2018-12-28,-0.035169,2738


## alpha191-20
$(CLOSE/DELAY(CLOSE,6)-1)*100$

In [25]:
alpha20 = (SecurityLatestValueHolder('closePrice') / SecurityShiftedValueHolder(6,
    SecurityLatestValueHolder('closePrice')) - 1) * 100

name = 'alpha20'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = alpha20.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

,alpha20,code
trade_date,,
2018-12-28,-6.382979,603999
2018-12-28,-4.496987,2729
2018-12-28,-4.909091,2740
2018-12-28,-5.210595,2739
2018-12-28,-4.152933,2738


## alpha191-21
$REGBETA(MEAN(CLOSE,6),SEQUENCE(6))$
- 缺少REGBETA:前 n 期样本 A 对 B 做回归所得回归系数
- 缺少SEQUENCE:生成 1~n 的等差序列

## alpha191-22
$SMEAN((CLOSE/MEAN(CLOSE,6)-1-DELAY(CLOSE/MEAN(CLOSE,6)-1,3)),12,1)$
- 缺少SMEAN 191原文中也未解释

In [26]:
#CLOSE/MEAN(CLOSE,6)
expression1 = SecurityLatestValueHolder('closePrice') / SecurityMovingAverage(6, 'closePrice')

#DELAY(expression1-1,3)
expression2 = SecurityShiftedValueHolder(3, expression1-1)

#expression1-1-expression2
expression3 = expression1 - 1 - expression2

name = 'alpha22'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = expression3.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

,alpha22,code
trade_date,,
2018-12-28,0.006871,603999
2018-12-28,-0.052045,2730
2018-12-28,-0.003468,2741
2018-12-28,-0.038176,2740
2018-12-28,-0.024338,2739


## alpha191-23
$SMA((CLOSE>DELAY(CLOSE,1)?STD(CLOSE,20):0),20,1) /
(SMA((CLOSE>DELAY(CLOSE,1)?STD(CLOSE,20):0),20,1)+SMA((CLOSE<=DELAY(CLOSE,1)?STD(CLOSE,20):0),20,1))
*100$
- 缺少SMA，用MA代替
- expression6 结果返回都是100

In [8]:

#CLOSE>DELAY(CLOSE,1)
expression1 = SecurityLatestValueHolder('closePrice') > SecurityDeltaValueHolder(
                1, SecurityLatestValueHolder('closePrice'))

#STD(CLOSE,20)
expression2 = SecurityMovingStandardDeviation(20, 'closePrice')

#(expression1?expression2:0)
expression3 = SecurityIIFValueHolder(expression1,expression2,0)

#CLOSE<=DELAY(CLOSE,1)
expression4 = SecurityLatestValueHolder('closePrice') <= SecurityDeltaValueHolder(
                1, SecurityLatestValueHolder('closePrice'))

#expression4?expression2:0
expression5 = SecurityIIFValueHolder(expression4,expression2,0)

expression6 = SecurityMovingAverage(20,expression3) / \
            (SecurityMovingAverage(20,expression3) + SecurityMovingAverage(20,expression5)) * 100
name = 'alpha23'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = expression6.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

/home/kerry/work/workenv/npyfin/lib/python3.6/site-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in greater
/home/kerry/work/workenv/npyfin/lib/python3.6/site-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in less_equal
/home/kerry/work/workenv/npyfin/lib/python3.6/site-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in true_divide


,alpha23,code
trade_date,,
2018-12-28,100.0,603999
2018-12-28,100.0,2729
2018-12-28,100.0,2740
2018-12-28,100.0,2739
2018-12-28,100.0,2738


## alpha191-24
$SMA(CLOSE-DELAY(CLOSE,5),5,1)$
- SMA缺少，用MA代替

In [104]:
expression1 = SecurityLatestValueHolder('closePrice') - SecurityShiftedValueHolder(
        5, SecurityLatestValueHolder('closePrice'))
expression2 = SecurityMovingAverage(5, expression1)

name = 'alpha24'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = expression2.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

,alpha24,code
trade_date,,
2018-12-28,-0.210,603999
2018-12-28,-0.404,2729
2018-12-28,-0.120,2740
2018-12-28,-0.622,2739
2018-12-28,-0.490,2738


## alpha191-25
$(-1*RANK(DELTA(CLOSE,7)*(1-RANK(DECAYLINEAR(VOLUME/MEAN(VOLUME,20),9)))))*(1+RANK(SUM(RET,250)))$
- SecurityXAverageValueHolder --移动加权平均
- 计算收益率 RET 暂时用 chgPct 涨跌幅代替

In [112]:
#DELTA(CLOSE,7)
expression1 = SecurityDeltaValueHolder(7, SecurityLatestValueHolder('closePrice'))
#VOLUME/MEAN(VOLUME,20)
expression2 = SecurityLatestValueHolder('turnoverVol')/SecurityMovingAverage(20, 'turnoverVol')
#DECAYLINEAR(expression2,9)
expression3 = SecurityXAverageValueHolder(9, expression2)
#1-RANK(expression3)
expression4 = 1 - CSPercentileSecurityValueHolder(expression3)
#RANK(expression1*expression4) * -1
expression5 = CSPercentileSecurityValueHolder(expression3 * expression4) * -1
#1 + RANK(SUM(RET,250))
expression6 = 1 + CSPercentileSecurityValueHolder(SecurityMovingSum(250,SecurityLatestValueHolder('chgPct')))
expression7 = expression5 * expression6
name = 'alpha25'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = expression7.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

,alpha25,code
trade_date,,
2018-12-28,-0.925072,603999
2018-12-28,-0.127201,2730
2018-12-28,-1.266748,2741
2018-12-28,-1.005846,2740
2018-12-28,-0.541175,2739


## alpha191-26
$(SUM(CLOSE,7)/7-CLOSE+CORR(VWAP,DELAY(CLOSE,5),230))$

In [113]:
#SUM(CLOSE,7)/7
expression1 = SecurityMovingSum(7, 'closePrice') / 7
#DELAY(CLOSE,5)
expression2 = SecurityShiftedValueHolder(5, SecurityLatestValueHolder('closePrice'))
#CORR(VWAP,expression2,230)
expression3 = SecurityMovingCorrelation(230, SecurityLatestValueHolder('vwap'), expression2)
#expression1-CLOSE+expression3
expression4 = expression1 - SecurityLatestValueHolder('closePrice') + expression3

name = 'alpha26'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = expression4.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

,alpha26,code
trade_date,,
2018-12-28,0.619746,603999
2018-12-28,1.238457,2729
2018-12-28,0.531979,2740
2018-12-28,0.851017,2739
2018-12-28,0.802353,2738


## alpha191-27
$WMA((CLOSE-DELTA(CLOSE,3))/DELAY(CLOSE,3)*100+(CLOSE-DELAY(CLOSE,6))/DELAY(CLOSE,6)*100,12)$
- 缺失WMA(A,n) 计算A前n期样本加权平均值权重为0.9i,，(i表示样本距离当前时点的间隔)
- 暂时用MA代替WMA

In [115]:
#(CLOSE-DELTA(CLOSE,3))/DELAY(CLOSE,3)
expression1 = (SecurityLatestValueHolder('closePrice') - SecurityDeltaValueHolder(
    3, SecurityLatestValueHolder('closePrice'))) / SecurityShiftedValueHolder(
    3, SecurityLatestValueHolder('closePrice'))

#(CLOSE-DELAY(CLOSE,6))/DELAY(CLOSE,6)
expression2 = (SecurityLatestValueHolder('closePrice') - SecurityShiftedValueHolder(
    6, SecurityLatestValueHolder('closePrice'))) / SecurityShiftedValueHolder(
    6, SecurityLatestValueHolder('closePrice'))

#expression1*100+expression2*100
expression3 = (expression1 + expression2) * 100

#WMA(expression1*100+expression2*100,12)
expression4 = SecurityMovingAverage(12, expression3)

name = 'alpha27'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = expression4.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

,alpha27,code
trade_date,,
2018-12-28,95.008292,603999
2018-12-28,98.028514,2729
2018-12-28,95.825000,2740
2018-12-28,99.102572,2739
2018-12-28,97.084025,2738


## alpha191-28
$3*SMA((CLOSE-TSMIN(LOW,9))/(TSMAX(HIGH,9)-TSMIN(LOW,9))*100,3,1)-2*SMA(SMA((CLOSE-TSMIN(LOW,9))/( MAX(HIGH,9)-TSMAX(LOW,9))*100,3,1),3,1)$
- 用MA代替SMA

In [116]:
#CLOSE-TSMIN(LOW,9)
expression1 = SecurityLatestValueHolder('closePrice') - SecurityMovingMin(9,
                SecurityLatestValueHolder('lowestPrice'))

#TSMAX(HIGH,9)-TSMIN(LOW,9)
expression2 = SecurityMovingMax(9,SecurityLatestValueHolder('highestPrice')) - \
            SecurityMovingMin(9,SecurityLatestValueHolder('lowestPrice'))

#MAX(HIGH,9)-TSMAX(LOW,9)
expression3 = SecurityMaximumValueHolder(SecurityLatestValueHolder('highestPrice'), 9 ) - \
            SecurityMovingMax(9,SecurityLatestValueHolder('highestPrice'))

#(expression1)/(expression2)*100
expression4 = expression1/expression2*100

#(expression1)/(expression3)
expression5 = expression1/expression3*100

#SMA(expression5,3,1)
expression6 = SecurityMovingAverage(3, expression5)

#3*SMA(expression4,3,1)-2*SMA(expression6,3,1)
expression7 = (3 * SecurityMovingAverage(3, expression4)) - (2 * SecurityMovingAverage(3, expression6))


name = 'alpha28'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = expression7.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

/home/kerry/work/workenv/npyfin/lib/python3.6/site-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in true_divide
/home/kerry/work/workenv/npyfin/lib/python3.6/site-packages/ipykernel_launcher.py:28: RuntimeWarning: divide by zero encountered in true_divide


,alpha28,code
trade_date,,
2018-12-28,39.398525,603999
2018-12-28,707.486711,2729
2018-12-28,21.008556,2740
2018-12-28,130.545824,2739
2018-12-28,155.616063,2738


## alpha191-29
$(CLOSE-DELAY(CLOSE,6))/DELAY(CLOSE,6)*VOLUME$

In [119]:
expression1 = (SecurityLatestValueHolder('closePrice') - SecurityShiftedValueHolder(
    6, SecurityLatestValueHolder('closePrice'))) / SecurityShiftedValueHolder(
    6, SecurityLatestValueHolder('closePrice')) * SecurityLatestValueHolder('turnoverVol')

name = 'alpha29'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = expression3.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

,alpha29,code
trade_date,,
2018-12-28,3.76,603999
2018-12-28,1.35,2730
2018-12-28,-0.57,2741
2018-12-28,3.31,2740
2018-12-28,-1.66,2739


## alpha191-31
$(CLOSE-MEAN(CLOSE,12))/MEAN(CLOSE,12)*100$

In [120]:
#CLOSE-MEAN(CLOSE,12)
expression1 = SecurityLatestValueHolder('closePrice') - SecurityMovingAverage(12, 'closePrice')
#expression1/MEAN(CLOSE,12)
expression2 = expression1 / SecurityMovingAverage(12, 'closePrice') * 100

name = 'alpha31'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = expression2.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

,alpha31,code
trade_date,,
2018-12-28,-4.457970,603999
2018-12-28,-1.217730,2730
2018-12-28,-2.341343,2741
2018-12-28,-4.750341,2740
2018-12-28,-4.721030,2739


## alpha191-32
$(-1*SUM(RANK(CORR(RANK(HIGH),RANK(VOLUME),3)),3))$

In [121]:
#RANK(HIGH)
expression1 = CSPercentileSecurityValueHolder(SecurityLatestValueHolder('highestPrice'))
#RANK(VOLUME)
expression2 = CSPercentileSecurityValueHolder(SecurityLatestValueHolder('turnoverVol'))
#CORR(expression1,expression2,3)
expression3 = SecurityMovingCorrelation(3,expression1,expression2)
#RANK(expression3)
expression4 = CSPercentileSecurityValueHolder(expression3)
#SUM(expression4,3)
expression5 = SecurityMovingSum(3, expression4) * -1
name = 'alpha32'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = expression5.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

,alpha32,code
trade_date,,
2018-12-28,-1.544570,603999
2018-12-28,-2.665875,2730
2018-12-28,-1.465656,2741
2018-12-28,-1.477306,2740
2018-12-28,-1.331506,2739


## alpha191-33
$(-1*TSMIN(LOW,5)+DELAY(TSMIN(LOW,5),5))*RANK((SUM(RET,240)-SUM(RET,20))/220)*TSRANK(VOLUME,5)$
- 用chgPct 代替RET

In [123]:
#-1*TSMIN(LOW,5)
expression1 = SecurityMovingMin(5, SecurityLatestValueHolder('lowestPrice')) * -1
#DELAY(TSMIN(LOW,5),5)
expression2 = SecurityShiftedValueHolder(5, SecurityMovingMin(5, SecurityLatestValueHolder('lowestPrice')))
#SUM(RET,240)
expression3 = SecurityMovingSum(240,SecurityLatestValueHolder('chgPct'))
#SUM(RET,20)
expression4 = SecurityMovingSum(240,SecurityLatestValueHolder('chgPct'))
#RANK((expression3-expression4)/220)
expression5 = CSPercentileSecurityValueHolder((expression3 - expression4) /220)
#TSRANK(VOLUME,5)
expression6 = SecurityMovingRank(5, SecurityLatestValueHolder('turnoverVol'))
#(expression1+expression2)*expression5*expression6
expression7 = (expression1 + expression2) * expression5 * expression6
name = 'alpha33'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = expression7.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

,alpha33,code
trade_date,,
2018-12-28,0.000000,603999
2018-12-28,0.465131,2729
2018-12-28,0.000000,2740
2018-12-28,2.940828,2739
2018-12-28,0.780220,2738


## alpha191-34
$MEAN(CLOSE,12)/CLOSE$

In [124]:
expression1 = SecurityMovingAverage(12, 'closePrice') / SecurityLatestValueHolder('closePrice')
name = 'alpha34'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = expression2.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

,alpha34,code
trade_date,,
2018-12-28,5.03,603999
2018-12-28,20.63,2729
2018-12-28,5.40,2740
2018-12-28,22.68,2739
2018-12-28,14.66,2738


## alpha191-35
$SMA(DELAY(CLOSE/DELAY(CLOSE,20),1),20,1)$

In [51]:
#CLOSE/DELAY(CLOSE,20)
expression1 =  SecurityLatestValueHolder('closePrice') / SecurityShiftedValueHolder(1, 
                                                    SecurityLatestValueHolder('closePrice'))
#DELAY(expression1,1)
expression2 = SecurityShiftedValueHolder(1,expression1)

#SMA(expression2,20,1)
expression3 = SecurityMovingAverage(20, expression2)


name = 'alpha35'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = expression3.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

,alpha35,code
trade_date,,
2018-12-28,1.000351,603999
2018-12-28,0.997312,2729
2018-12-28,0.994659,2740
2018-12-28,0.999551,2739
2018-12-28,0.996494,2738


## alpha191-36
$RANK(SUM(CORR(RANK(VOLUME),RANK(VWAP),6),2))$

In [125]:
# RANK(VOLUME)
expression1 = CSPercentileSecurityValueHolder(SecurityLatestValueHolder('turnoverVol'))
#RANK(VWAP)
expression2 = CSPercentileSecurityValueHolder(SecurityLatestValueHolder('vwap'))
#CORR(expression1,expression2,6)
expression3 = SecurityMovingCorrelation(6, expression1, expression2)
#RANK(SUM(expression3,2))
expression4 = CSPercentileSecurityValueHolder(SecurityMovingSum(2, expression3))

name = 'alpha34'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = expression4.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

,alpha34,code
trade_date,,
2018-12-28,0.280924,603999
2018-12-28,0.996901,2730
2018-12-28,0.203719,2741
2018-12-28,0.737109,2740
2018-12-28,0.267681,2739


# alpha191-37
$(-1*RANK(SUM(OPEN,5)*SUM(RET,5)-DELAY(SUM(OPEN,5)*SUM(RET,5),10)))$
- 用chgPct 代替RET

In [126]:
#SUM(OPEN,5)
expression1 = SecurityMovingSum(5, SecurityLatestValueHolder('openPrice'))
#SUM(RET,5)
expression2 = SecurityMovingSum(5, SecurityLatestValueHolder('chgPct'))
#DELAY(expression1*expression2,10)
expression3 =SecurityShiftedValueHolder(10, expression1+expression2)
#-1*RANK(expression1*expression2-expression3)
expression4 = CSPercentileSecurityValueHolder(expression1*expression2-expression3) * -1
name = 'alpha37'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = expression4.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

,alpha37,code
trade_date,,
2018-12-28,-0.706960,603999
2018-12-28,-0.116934,2729
2018-12-28,-0.669766,2740
2018-12-28,-0.090730,2739
2018-12-28,-0.214427,2738


# alpha191-38
$((SUM(HIGH,20)/20)<HIGH)?(-1*DELTA(HIGH,2)):0$

In [133]:
#(-1*DELTA(HIGH,2))
expression1 = SecurityDeltaValueHolder(2, SecurityLatestValueHolder('highestPrice')) * -1
#(SUM(HIGH,20)/20)
expression2 = SecurityMovingSum(20, SecurityLatestValueHolder('highestPrice')) / 20
#expression2<HIGH
expression3 = expression2 < SecurityLatestValueHolder('highestPrice')
#expression3?expression1:0
expression4 = SecurityIIFValueHolder(expression3, expression1, 0)
name = 'alpha38'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = expression4.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

,alpha38,code
trade_date,,
2018-12-28,0.0,603999
2018-12-28,0.0,2729
2018-12-28,0.0,2740
2018-12-28,0.0,2739
2018-12-28,0.0,2738


## alpha191-39
$(RANK(DECAYLINEAR(DELTA(CLOSE,2),8))-RANK(DECAYLINEAR(CORR(VWAP*0.3+OPEN*0.7,SUM(MEAN(VOLUME,180),37),14),12)))*-1$

In [134]:
#DELTA(CLOSE,2)
expression1 = SecurityDeltaValueHolder(2, SecurityLatestValueHolder('closePrice'))
#RANK(DECAYLINEAR(expression1,8))
expression2 = CSPercentileSecurityValueHolder(SecurityXAverageValueHolder(8, expression1))
#VWAP*0.3+OPEN*0.7
expression3 = SecurityLatestValueHolder('vwap') * 0.3 + SecurityLatestValueHolder('openPrice') * 0.7
#SUM(MEAN(VOLUME,180),37)
expression4 = SecurityMovingSum(37,SecurityMovingAverage(180, 'turnoverVol'))
#CORR(expression3,expression4,14)
expression5 = SecurityMovingCorrelation(14, expression3, expression4)
#RANK(DECAYLINEAR(expression5,12))
expression6 = CSPercentileSecurityValueHolder(
                SecurityXAverageValueHolder(12, expression5))
#(expression2-expression6) * -1
expression7 = (expression2 * expression6) * -1

name = 'alpha39'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = expression7.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

,alpha39,code
trade_date,,
2018-12-28,-0.151666,603999
2018-12-28,-0.318551,2730
2018-12-28,-0.070034,2741
2018-12-28,-0.084458,2740
2018-12-28,-0.018479,2739


## alpha191-40
$SUM(CLOSE>DELAY(CLOSE,1)?VOLUME:0,26)/SUM(CLOSE<=DELAY(CLOSE,1)?VOLUME:0,26)*100$
- expression7 = expression3/expression6 相除为空
- expression7 = expression3 + expression6 相加为0

In [7]:
#CLOSE>DELAY(CLOSE,1)
expression1 = SecurityLatestValueHolder('closePrice') > \
            SecurityShiftedValueHolder(1,SecurityLatestValueHolder('closePrice'))

#expression1?VOLUME:0
expression2 = SecurityIIFValueHolder(expression1,SecurityLatestValueHolder('turnoverVol'), 0)

#SUM(expression2,26)
expression3 = SecurityMovingSum(26, expression2)

#CLOSE<=DELAY(CLOSE,1)
expression4 = SecurityLatestValueHolder('closePrice') <= \
            SecurityShiftedValueHolder(1,SecurityLatestValueHolder('closePrice'))

#expression4?VOLUME:0
expression5 = SecurityIIFValueHolder(expression4,SecurityLatestValueHolder('turnoverVol'), 0)
#SUM(expression5,26)
expression6 = SecurityMovingSum(26, expression5)
#expression3/expression6*100
expression7 = expression3/expression6*100
name = 'alpha40'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = expression7.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

/home/kerry/work/workenv/npyfin/lib/python3.6/site-packages/ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in greater
/home/kerry/work/workenv/npyfin/lib/python3.6/site-packages/ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in less_equal
/home/kerry/work/workenv/npyfin/lib/python3.6/site-packages/ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in true_divide
/home/kerry/work/workenv/npyfin/lib/python3.6/site-packages/ipykernel_launcher.py:23: RuntimeWarning: divide by zero encountered in true_divide


,alpha40,code
trade_date,,
2018-12-28,51.522969,603999
2018-12-28,88.934989,2729
2018-12-28,64.970652,2740
2018-12-28,124.376637,2739
2018-12-28,63.820612,2738


## alpha191-41
$RANK(MAX(DELTA(VWAP,3),5))*-1$

In [149]:
expression1 = SecurityDeltaValueHolder(3,SecurityLogValueHolder('vwap'))
expression2 = CSPercentileSecurityValueHolder(SecurityMovingMax(5, expression1)) * -1
name = 'alpha41'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = expression2.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

,alpha41,code
trade_date,,
2018-12-28,-0.231333,603999
2018-12-28,-0.978867,2730
2018-12-28,-0.427163,2741
2018-12-28,-0.267963,2740
2018-12-28,-0.627501,2739


## alpha191-42
$(-1*RANK(STD(HIGH,10)))*CORR(HIGH,VOLUME,10)$

In [150]:
#STD(HIGH,10)
expression1 = SecurityMovingStandardDeviation(10, 'highestPrice')
#-1*RANK(expression1)
expression2 = CSPercentileSecurityValueHolder(expression1) * -1
#CORR(HIGH,VOLUME,10)
expression3 = SecurityMovingCorrelation(10,SecurityLatestValueHolder('highestPrice'),
                                        SecurityLatestValueHolder('turnoverVol'))
#expression2*expression3
expression4 = expression2 * expression3

name = 'alpha42'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = expression4.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

,alpha42,code
trade_date,,
2018-12-28,-0.030855,603999
2018-12-28,0.355061,2729
2018-12-28,-0.141281,2740
2018-12-28,0.145803,2739
2018-12-28,-0.024169,2738


## alpha191-43
$(SUM(CLOSE>DELAY(CLOSE,1)?VOLUME:(CLOSE<DELAY(CLOSE,1)?-VOLUME:0),6))$

In [151]:
#CLOSE>DELAY(CLOSE,1)
expression1 = SecurityLatestValueHolder('closePrice') > \
        SecurityShiftedValueHolder(1,SecurityLatestValueHolder('closePrice'))

#CLOSE<DELAY(CLOSE,1)
expression2 = SecurityLatestValueHolder('closePrice') < \
        SecurityShiftedValueHolder(1,SecurityLatestValueHolder('closePrice'))

#expression2?-VOLUME:0
expression3 = SecurityIIFValueHolder(expression2, -SecurityLatestValueHolder('turnoverVol'), 0)

#expression1?VOLUME:expression3
expression4 = SecurityIIFValueHolder(expression1, SecurityLatestValueHolder('turnoverVol'), 
                                     expression3)
#SUM(expression4,6)
expression5 = SecurityMovingSum(6, expression4)

name = 'alpha43'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = expression5.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

/home/kerry/work/workenv/npyfin/lib/python3.6/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in greater
/home/kerry/work/workenv/npyfin/lib/python3.6/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in less


,alpha43,code
trade_date,,
2018-12-28,-8764440.0,603999
2018-12-28,4082687.0,2730
2018-12-28,-697669.0,2741
2018-12-28,-1714147.0,2740
2018-12-28,1110667.0,2739


## alpha191-44
$(SUM(CLOSE>DELAY(CLOSE,1)?VOLUME:(CLOSE<DELAY(CLOSE,1)?-VOLUME:0),6))$

In [152]:
#CLOSE>DELAY(CLOSE,1)
expression1 = SecurityLatestValueHolder('closePrice') > \
        SecurityShiftedValueHolder(1,SecurityLatestValueHolder('closePrice'))
#CLOSE<DELAY(CLOSE,1)
expression2 = SecurityLatestValueHolder('closePrice') < \
        SecurityShiftedValueHolder(1,SecurityLatestValueHolder('closePrice'))
#expression2?-VOLUME:0
expression3 = SecurityIIFValueHolder(expression2, -SecurityLatestValueHolder('turnoverVol'),
                                    0)
#expression1?VOLUME:expression3
expression4 = SecurityIIFValueHolder(expression1, SecurityLatestValueHolder('turnoverVol'), expression3)
#SUM(expression4,6)
expression5 = SecurityMovingSum(6, expression4)
name = 'alpha44'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = expression5.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).loc['2018-12-28'].head(5)

/home/kerry/work/workenv/npyfin/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in greater
  app.launch_new_instance()
/home/kerry/work/workenv/npyfin/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in less
  app.launch_new_instance()


,alpha44,code
trade_date,,
2018-12-28,-8764440.0,603999
2018-12-28,4082687.0,2730
2018-12-28,-697669.0,2741
2018-12-28,-1714147.0,2740
2018-12-28,1110667.0,2739


# alpha191-45
$(RANK(DELTA(CLOSE*0.6+OPEN*0.4,1))*RANK(CORR(VWAP,MEAN(VOLUME,150),15)))$

In [153]:
#CLOSE*0.6+OPEN*0.4
expression1 = SecurityLatestValueHolder('closePrice') * 0.6 + SecurityLatestValueHolder('openPrice') * 0.4
#DELTA(expression1,1)
expression2 = SecurityDeltaValueHolder(1, expression1)
#RANK(expression2)
expression3 = CSPercentileSecurityValueHolder(expression2)
#MEAN(VOLUME,150)
expression4 = SecurityMovingAverage(150,'turnoverVol')
#RANK(CORR(VWAP,expression4,15))
expression5 = CSPercentileSecurityValueHolder(
                SecurityMovingCorrelation(5, SecurityLatestValueHolder('vwap'), expression4))
#expression3*expression5
expression6 = expression3 * expression5
name = 'alpha45'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = expression6.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).loc['2018-12-28'].head(5)

,alpha45,code
trade_date,,
2018-12-28,0.299632,603999
2018-12-28,0.088643,2729
2018-12-28,0.412799,2740
2018-12-28,0.086874,2739
2018-12-28,0.632736,2738


## alpha191-46
$(MEAN(CLOSE,3)+MEAN(CLOSE,6)+MEAN(CLOSE,12)+MEAN(CLOSE,24))/(4*CLOSE)$

In [154]:
#MEAN(CLOSE,3)
expression1 = SecurityMovingAverage(3,'closePrice')
#MEAN(CLOSE,6)
expression2 = SecurityMovingAverage(6,'closePrice')
#MEAN(CLOSE,12)
expression3 = SecurityMovingAverage(12,'closePrice')
#MEAN(CLOSE,24)
expression4 = SecurityMovingAverage(24,'closePrice')
#(expression1+expression2+expression3+expression4)/(4*CLOSE)
expression5 = (expression1 + expression2 + expression3 + expression4)/\
            (4 * SecurityLatestValueHolder('closePrice'))

name = 'alpha46'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = expression6.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

,alpha46,code
trade_date,,
2018-12-28,0.300702,603999
2018-12-28,0.110683,2730
2018-12-28,0.143235,2741
2018-12-28,0.417696,2740
2018-12-28,0.082223,2739


## alpha191-47
$SMA((TSMAX(HIGH,6)-CLOSE)/(TSMAX(HIGH,6)-TSMIN(LOW,6))*100,9,1)$
- MA 代替SMA

In [155]:
#TSMAX(HIGH,6)-CLOSE
expression1 = SecurityMovingMax(6, 'highestPrice') - SecurityLatestValueHolder('closePrice')
#TSMAX(HIGH,6)-TSMIN(LOW,6)
expression2 = SecurityMovingMax(6, 'highestPrice') - SecurityMovingMin(6, 'lowestPrice')
#expression1/expression2*100
expression3 = expression1/expression2*100
#SMA(expression3,9,1)
expression4 = SecurityMovingAverage(9, expression3)

name = 'alpha47'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = expression4.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).loc['2018-12-28'].head(5)

/home/kerry/work/workenv/npyfin/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in true_divide
  if sys.path[0] == '':


,alpha47,code
trade_date,,
2018-12-28,74.111291,603999
2018-12-28,57.360972,2729
2018-12-28,78.832727,2740
2018-12-28,71.427247,2739
2018-12-28,68.676492,2738


## alpha191-48
$-1*RANK(SIGN(CLOSE-DELAY(CLOSE,1))+SIGN(DELAY(CLOSE,1)-DELAY(CLOSE,2))+SIGN(DELAY(CLOSE,2)-DELAY(CLOSE,3)))*SUM(VOLUME,5)/SUM(VOLUME,20)$

In [156]:

#CLOSE-DELAY(CLOSE,1)
expression1 = SecurityLatestValueHolder('closePrice') - SecurityShiftedValueHolder(1,
        SecurityLatestValueHolder('closePrice'))

#SIGN(expression1)
expression2 = SecuritySignValueHolder(expression1)

#DELAY(CLOSE,1)-DELAY(CLOSE,2)
expression3 = SecurityShiftedValueHolder(1, SecurityLatestValueHolder('closePrice')) - \
             SecurityShiftedValueHolder(2, SecurityLatestValueHolder('closePrice'))

#SIGN(expression3)
expression4 = SecuritySignValueHolder(expression3)

#DELAY(CLOSE,2)-DELAY(CLOSE,3)
expression5 = SecurityShiftedValueHolder(2, SecurityLatestValueHolder('closePrice')) - \
             SecurityShiftedValueHolder(3, SecurityLatestValueHolder('closePrice'))

#SIGN(expression5)
expression6 = SecuritySignValueHolder(expression5)

#-1*RANK(expression2+expression4+expression6)*SUM(VOLUME,5)/SUM(VOLUME,20)
expression7 = -1 * CSPercentileSecurityValueHolder(expression2 + expression4 + expression6) * \
            SecurityMovingSum(5, 'turnoverVol') / SecurityMovingSum(20, 'turnoverVol') 

name = 'alpha48'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = expression7.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).loc['2018-12-28'].head(5)

,alpha48,code
trade_date,,
2018-12-28,-0.081246,603999
2018-12-28,-0.197135,2730
2018-12-28,-0.158921,2741
2018-12-28,-0.016401,2740
2018-12-28,-0.088138,2739


## alpha191-49
$ SUM(HIGH+LOW>=DELAY(HIGH,1)+DELAY(LOW,1)?0:MAX(ABS(HIGH-DELAY(HIGH,1)),ABS(LOW-DELAY(LOW,1))),12)/
  (SUM(HIGH+LOW>=DELAY(HIGH,1)+DELAY(LOW,1)?0:MAX(ABS(HIGH-DELAY(HIGH,1)),ABS(LOW-DELAY(LOW,1))),12)+
  SUM(HIGH+LOW<=DELAY(HIGH,1)+DELAY(LOW,1)?0:MAX(ABS(HIGH-DELAY(HIGH,1)),ABS(LOW-DELAY(LOW,1))),12))$
  
- 相除为空

In [8]:
#DELAY(HIGH,1)
expression1 = SecurityShiftedValueHolder(1, SecurityLatestValueHolder('highestPrice'))
#DELAY(LOW,1)
expression2 = SecurityShiftedValueHolder(1, SecurityLatestValueHolder('lowestPrice'))
#HIGH-expression1
expression3 = SecurityLatestValueHolder('highestPrice') - expression1
#LOW-expression2
expression4 = SecurityLatestValueHolder('lowestPrice') - expression2
#HIGH+LOW
expression5 = SecurityLatestValueHolder('highestPrice') + SecurityLatestValueHolder('lowestPrice')
#ABS(expression3)
expression6 = SecurityAbsValueHolder(expression3)
#ABS(expression4)
expression7 = SecurityAbsValueHolder(expression4)
#MAX(expression6,expression7)
expression8 = SecurityMaximumValueHolder(expression6, expression7)
#expression1+expression2
expression9 = expression1 + expression2
#expression5>=expression9
expression10 = expression5>=expression9
#expression5<=expression9
expression11 = expression5<=expression9
#expression10?0:expression8
expression12 = SecurityIIFValueHolder(expression10,0,expression8)
#expression11?0:expression8
expression13 = SecurityIIFValueHolder(expression11,0,expression8)
# SUM(expression12,12)/(SUM(expression12,12)+SUM(expression13,12))
expression14 = (SecurityMovingSum(10, expression12)) / (
    SecurityMovingSum(10, expression12) + SecurityMovingSum(10, expression13))

name = 'alpha49'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date').sort_values(
    by='trade_date',ascending=False)
result = expression14.transform(df,name=name,category_field='code')
result.sort_values(by='alpha49', ascending=False).head(5)

/home/kerry/work/workenv/npyfin/lib/python3.6/site-packages/ipykernel_launcher.py:34: RuntimeWarning: invalid value encountered in greater_equal
/home/kerry/work/workenv/npyfin/lib/python3.6/site-packages/ipykernel_launcher.py:34: RuntimeWarning: invalid value encountered in less_equal
/home/kerry/work/workenv/npyfin/lib/python3.6/site-packages/ipykernel_launcher.py:34: RuntimeWarning: invalid value encountered in true_divide


,alpha49,code
trade_date,,
2018-10-31,1.0,2575
2018-12-27,1.0,600061
2018-12-24,1.0,300619
2018-12-27,1.0,600056
2018-12-27,1.0,600053


## alpha191-50
$SUM(HIGH+LOW<=DELAY(HIGH,1)+DELAY(LOW,1)?0:MAX(ABS(HIGH-DELAY(HIGH,1)),ABS(LOW-DELAY(LOW,1))),12)/
(SUM(HIGH+LOW<=DELAY(HIGH,1)+DELAY(LOW,1)?0:MAX(ABS(HIGH-DELAY(HIGH,1)),ABS(LOW-DELAY(LOW,1))),12)
+SUM(HIGH+LOW>=DELAY(HIGH,1)+DELAY(LOW,1)?0:MAX(ABS(HIGH-DELAY(HIGH,1)),ABS(LOW-DELAY(LOW,1))),12))
-SUM(HIGH+LOW>=DELAY(HIGH,1)+DELAY(LOW,1)?0:MAX(ABS(HIGH-DELAY(HIGH,1)),ABS(LOW-DELAY(LOW,1))),12)/
(SUM(HIGH+LOW>=DELAY(HIGH,1)+DELAY(LOW,1)?0:MAX(ABS(HIGH-DELAY(HIGH,1)),ABS(LOW-DELAY(LOW,1))),12)
+SUM(HIGH+LOW<=DELAY(HIGH,1)+DELAY(LOW,1)?0:MAX(ABS(HIGH-DELAY(HIGH,1)),ABS(LOW-DELAY(LOW,1))),12))$

- 相除为空

In [9]:
# expression10/expression10+expression11-expression11/expression11 + expression10

expression1 = SecurityShiftedValueHolder(1, SecurityLatestValueHolder('highestPrice'))
expression2 = SecurityShiftedValueHolder(1, SecurityLatestValueHolder('lowestPrice'))
expression3 = expression1+expression2
expression4 = SecurityLatestValueHolder('highestPrice') - \
                SecurityShiftedValueHolder(1, SecurityLatestValueHolder('highestPrice'))
expression5 = SecurityLatestValueHolder('lowestPrice') - \
                SecurityShiftedValueHolder(1, SecurityLatestValueHolder('lowestPrice'))
expression6 = SecurityMaximumValueHolder(SecurityAbsValueHolder(expression4),
                                         SecurityAbsValueHolder(expression5))
expression7 = SecurityLatestValueHolder('highestPrice') + SecurityLatestValueHolder('lowestPrice')
expression8 = SecurityIIFValueHolder(expression7<=expression3,0, expression6)
expression9 = SecurityIIFValueHolder(expression7>=expression3,0, expression6)
expression10 = SecurityMovingSum(12, expression8) 
expression11 = SecurityMovingSum(12, expression9)
expression12 = expression10 / (expression10 + expression11) - expression11 / (expression11 + expression10)
name = 'alpha50'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date'
                                ).sort_values(by='trade_date',ascending=False)

result = expression12.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

/home/kerry/work/workenv/npyfin/lib/python3.6/site-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in less_equal
/home/kerry/work/workenv/npyfin/lib/python3.6/site-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in greater_equal
/home/kerry/work/workenv/npyfin/lib/python3.6/site-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in true_divide


,alpha50,code
trade_date,,
2018-12-27,1.0,2737
2018-12-27,1.0,603988
2018-12-27,1.0,603955
2018-12-27,1.0,603779
2018-12-27,1.0,603985


## alpha191-51
$SUM(((HIGH+LOW)<=(DELAY(HIGH,1)+DELAY(LOW,1))?0:MAX(ABS(HIGH-DELAY(HIGH,1)),ABS(LOW-DELAY(LOW,1)))),12)/
(SUM(((HIGH+LOW)<=(DELAY(HIGH,1)+DELAY(LOW,1))?0:MAX(ABS(HIGH-DELAY(HIGH,1)),ABS(LOW-DELAY(LOW,1)))),12)
+SUM(((HIGH+LOW)>=(DELAY(HIGH,1)+DELAY(LOW,1))?0:MAX(ABS(HIGH-DELAY(HIGH,1)),ABS(LOW-DELAY(LOW,1)))),12))$

- 相除为空

In [10]:
expression1 = SecurityShiftedValueHolder(1, SecurityLatestValueHolder('highestPrice')) + \
                SecurityShiftedValueHolder(1, SecurityLatestValueHolder('lowestPrice'))
expression2 = SecurityAbsValueHolder(SecurityLatestValueHolder('highestPrice') - \
                SecurityShiftedValueHolder(1, SecurityLatestValueHolder('highestPrice')))
expression3 = SecurityAbsValueHolder(SecurityLatestValueHolder('lowestPrice') - \
                SecurityShiftedValueHolder(1, SecurityLatestValueHolder('lowestPrice')))
expression4 = SecurityMaximumValueHolder(expression2,expression3)
expression5 = SecurityLatestValueHolder('highestPrice') + SecurityLatestValueHolder('lowestPrice')
expression6 = SecurityIIFValueHolder(expression5<=expression1,0,expression4)
expression7 = SecurityIIFValueHolder(expression5>=expression1,0,expression4)
expression8 = expression6 / (expression6 + expression7)
name = 'alpha51'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = expression8.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

/home/kerry/work/workenv/npyfin/lib/python3.6/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in less_equal
  
/home/kerry/work/workenv/npyfin/lib/python3.6/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in greater_equal
  
/home/kerry/work/workenv/npyfin/lib/python3.6/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in true_divide
  


,alpha51,code
trade_date,,
2018-12-28,0.0,603999
2018-12-28,0.0,2732
2018-12-28,0.0,2743
2018-12-28,0.0,2742
2018-12-28,0.0,2741


## alpha191-52
$SUM(MAX(0,HIGH-DELAY((HIGH+LOW+CLOSE)/3,1)),26)/SUM(MAX(0,DELAY((HIGH+LOW+CLOSE)/3,1)-L),26)*100$

In [11]:
#(HIGH+LOW+CLOSE)/3
expression1 = (SecurityLatestValueHolder('highestPrice') + SecurityLatestValueHolder('highestPrice') + \
            SecurityLatestValueHolder('highestPrice')) / 3

#HIGH-DELAY(expression1,1)
expression2 = SecurityLatestValueHolder('highestPrice') - SecurityShiftedValueHolder(1,expression1)

#MAX(0,expression2)
expression3 = SecurityMaximumValueHolder(0, expression2)

#MAX(0,DELAY(expression1,1)-L)
expression4 = SecurityMaximumValueHolder(0, expression1 - SecurityLatestValueHolder('lowestPrice'))

#SUM(expression3,26)/SUM(expression4,26)*100
expression5 = SecurityMovingSum(26,expression3) / SecurityMovingSum(26,expression4) * 100
name = 'alpha52'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = expression5.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

/home/kerry/work/workenv/npyfin/lib/python3.6/site-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide
/home/kerry/work/workenv/npyfin/lib/python3.6/site-packages/ipykernel_launcher.py:18: RuntimeWarning: divide by zero encountered in true_divide


,alpha52,code
trade_date,,
2018-12-28,27.853261,603999
2018-12-28,31.743666,2730
2018-12-28,18.550228,2741
2018-12-28,15.843271,2740
2018-12-28,20.820734,2739


## alpha191-53
$COUNT(CLOSE>DELAY(CLOSE,1),12)/12*100$
- 缺少count函数: COUNT(condition, n)  计算前 n 期满足条件 condition 的样本个数COUNT(condition, n)  

In [12]:
expression1 = SecurityLatestValueHolder('closePrice') > SecurityShiftedValueHolder(
    1,SecurityLatestValueHolder('closePrice'))

name = 'alpha53'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = expression1.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

/home/kerry/work/workenv/npyfin/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in greater
  


,alpha53,code
trade_date,,
2018-12-28,1.0,603999
2018-12-28,0.0,2730
2018-12-28,1.0,2741
2018-12-28,0.0,2740
2018-12-28,1.0,2739


## alpha191-54
$(-1*RANK(STD(ABS(CLOSE-OPEN))+CLOSE-OPEN+CORR(CLOSE,OPEN,10)))$
- 缺少横截面STD

In [13]:
#ABS(CLOSE-OPEN)
expression1 = SecurityAbsValueHolder(SecurityLatestValueHolder('closePrice') - \
                                     SecurityLatestValueHolder('openPrice'))

#STD(expression1)
expression2 = expression1
#CORR(CLOSE,OPEN,10)
expression3 = SecurityMovingCorrelation(5, SecurityLatestValueHolder('closePrice'),
                                        SecurityLatestValueHolder('openPrice'))
#-1*RANK(expression2+CLOSE-OPEN+expression3)
expression4 = -1 * CSPercentileSecurityValueHolder(expression2 + SecurityLatestValueHolder('closePrice') - \
            SecurityLatestValueHolder('openPrice') + expression3)

name = 'alpha55'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = expression4.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

,alpha55,code
trade_date,,
2018-12-28,-0.586081,603999
2018-12-28,-0.186531,2729
2018-12-28,-0.798817,2740
2018-12-28,-0.967033,2739
2018-12-28,-0.660468,2738


## alpha191-55

$SUM(16*(CLOSE+(CLOSE-OPEN)/2-DELAY(OPEN,1))/((ABS(HIGH-DELAY(CLOSE,1))>ABS(LOW-DELAY(CLOSE,1)) & ABS(HIGH-DELAY(CLOSE,1))>ABS(HIGH-DELAY(LOW,1)) ? ABS(HIGH-DELAY(CLOSE,1))+ABS(LOW-DELAY(CLOSE,1))/2+ABS(DELAY(CLOSE,1)-DELAY(OPEN,1))/4:(ABS(LOW-DELAY(CLOSE,1))>ABS(HIGH-DELAY(LOW,1)) & ABS(LOW-DELAY(CLOSE,1))>ABS(HIGH-DELAY(CLOSE,1)) ? ABS(LOW-DELAY(CLOSE,1))+ABS(HIGH-DELAY(CLOSE,1))/2+ABS(DELAY(CLOSE,1)-DELAY(OPEN,1))/4: ABS(HIGH-DELAY(LOW,1))+ABS(DELAY(CLOSE,1)-DELAY(OPEN,1))/4)))*MAX(ABS(HIGH-DELAY(CLOSE,1)),ABS(LOW-DELAY(CLOSE,1))),20)$

In [14]:
#SUM(expression8,20)$


#ABS(HIGH-DELAY(CLOSE,1))
expression1 = SecurityAbsValueHolder(SecurityLatestValueHolder('highestPrice') - SecurityShiftedValueHolder(
        1,SecurityLatestValueHolder('closePrice')))

#ABS(LOW-DELAY(CLOSE,1))
expression2 = SecurityAbsValueHolder(SecurityLatestValueHolder('lowestPrice') - SecurityShiftedValueHolder(
        1,SecurityLatestValueHolder('closePrice')))

#ABS(HIGH-DELAY(LOW,1))
expression3 = SecurityAbsValueHolder(SecurityLatestValueHolder('highestPrice') - SecurityShiftedValueHolder(
        1,SecurityLatestValueHolder('closePrice')))

#ABS(DELAY(CLOSE,1)-DELAY(OPEN,1))
expression4 = SecurityAbsValueHolder(SecurityShiftedValueHolder(1,
        SecurityLatestValueHolder('closePrice')) - SecurityShiftedValueHolder(1,
        SecurityLatestValueHolder('openPrice')))

#CLOSE+(CLOSE-OPEN)/2-DELAY(OPEN,1)
expression5 = SecurityLatestValueHolder('closePrice') + (
        SecurityLatestValueHolder('closePrice') - SecurityLatestValueHolder('openPrice')) / 2 - \
        SecurityShiftedValueHolder(1,SecurityLatestValueHolder('openPrice'))

#(expression2>expression3 & expression2>expression1?expression2+expression1/2+expression4/4:
#expression3+expression4/4)
expression6 = SecurityIIFValueHolder((expression2 > expression3) & (expression2 > expression1),
                                     expression2 + expression1/2 + expression4/4,
                                     expression3 + expression4/4)

#(expression1>expression2 & expression1>expression3 ? expression1+expression2/2+expression4/4:expression6)
expression7 = SecurityIIFValueHolder((expression2 > expression3) & (expression2 > expression1),
                                     expression2 + expression1/2 + expression4/4,
                                        expression3+expression4/4)

#16*(expression5)/(expression7)*MAX(expression1,expression2)
expression8 = 16 *expression5 / expression7 * SecurityMaximumValueHolder(expression1,expression2)
expression9 = SecurityMovingSum(20, expression8) 

name = 'alpha55'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = expression8.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

/home/kerry/work/workenv/npyfin/lib/python3.6/site-packages/ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in greater


,alpha55,code
trade_date,,
2018-12-28,-2.125283,603999
2018-12-28,-12.142776,2729
2018-12-28,-2.000000,2740
2018-12-28,-10.822724,2739
2018-12-28,-1.980000,2738


## alpha191-56
$RANK(OPEN-TSMIN(OPEN,12))<RANK(RANK(CORR(SUM((HIGH +LOW)/2,19),SUM(MEAN(VOLUME,40),19),13))^5)$

In [173]:
#RANK(expression1)<RANK(expression6)

#OPEN-TSMIN(OPEN,12)
expression1 = SecurityLatestValueHolder('openPrice') - SecurityMovingMin(12,SecurityLatestValueHolder('openPrice'))

#SUM((HIGH +LOW)/2,19)
expression2 = SecurityMovingSum(9, 
            (SecurityLatestValueHolder('highestPrice') + SecurityLatestValueHolder('lowestPrice'))/2)

#SUM(MEAN(VOLUME,40),19)
expression3 = SecurityMovingSum(9,SecurityMovingAverage(10, 'turnoverVol'))

#CORR(expression2,expression3,13)
expression4 = SecurityMovingCorrelation(3, expression2,expression3)

#RANK(expression1)
expression5 = CSPercentileSecurityValueHolder(expression1)

#RANK(expression4)^5
expression6 = SecurityPowValueHolder(CSPercentileSecurityValueHolder(expression4),5)

expression7 = expression5 < CSPercentileSecurityValueHolder(expression6)
name = 'alpha56'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = expression7.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

/home/kerry/work/workenv/npyfin/lib/python3.6/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in less


,alpha56,code
trade_date,,
2018-12-28,1.0,603999
2018-12-28,0.0,2730
2018-12-28,1.0,2741
2018-12-28,1.0,2740
2018-12-28,1.0,2739


## alpha191-57
$SMA((CLOSE-TSMIN(LOW,9))/(TSMAX(HIGH,9)-TSMIN(LOW,9))*100,3,1)$
- MA代替SMA

In [174]:
#SMA(expression1/expression2*100,3,1)
expression1 = SecurityLatestValueHolder('closePrice') - SecurityMovingMin(
    9,SecurityLatestValueHolder('lowestPrice'))
#(TSMAX(HIGH,9)-TSMIN(LOW,9))
expression2 = SecurityMovingMax(9,SecurityLatestValueHolder('highestPrice')) \
        - SecurityMovingMin(9,SecurityLatestValueHolder('lowestPrice'))
expression3 = expression1 / expression2 * 100
expression4 = SecurityMovingAverage(9, expression3)
name = 'alpha57'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = expression4.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

/home/kerry/work/workenv/npyfin/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in true_divide
  # This is added back by InteractiveShellApp.init_path()


,alpha57,code
trade_date,,
2018-12-28,16.855920,603999
2018-12-28,35.938372,2729
2018-12-28,15.000385,2740
2018-12-28,29.141028,2739
2018-12-28,26.031979,2738


## alpha191-58
$COUNT(CLOSE>DELAY(CLOSE,1),20)/20*100$
- 缺少count

In [178]:
#COUNT(CLOSE>DELAY(CLOSE,1),20)/20*100
expression1 = SecurityLatestValueHolder('closePrice') - SecurityShiftedValueHolder(1,
                                    SecurityLatestValueHolder('closePrice'))
name = 'alpha58'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = expression1.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

,alpha58,code
trade_date,,
2018-12-28,0.01,603999
2018-12-28,0.18,2729
2018-12-28,-0.02,2740
2018-12-28,0.57,2739
2018-12-28,0.40,2738


## alpha191-59
$SUM((CLOSE=DELAY(CLOSE,1)?0:CLOSE-(CLOSE>DELAY(CLOSE,1)?MIN(LOW,DELAY(CLOSE,1)):MAX(HIGH,DELAY(CLOSE,1)))),20)$

In [179]:

#MAX(HIGH,DELAY(CLOSE,1))
expression1 = SecurityMaximumValueHolder(SecurityLatestValueHolder('highestPrice'),
                    SecurityShiftedValueHolder(1,SecurityLatestValueHolder('closePrice')))

#MIN(LOW,DELAY(CLOSE,1))
expression2 = SecurityMinimumValueHolder(SecurityLatestValueHolder('lowestPrice'),
                    SecurityShiftedValueHolder(1,SecurityLatestValueHolder('closePrice')))

#CLOSE>DELAY(CLOSE,1)
expression3 = SecurityLatestValueHolder('closePrice') > \
        SecurityShiftedValueHolder(1,SecurityLatestValueHolder('closePrice'))

#(expression3?expression2:expression1)
expression4 = SecurityIIFValueHolder(expression3,expression2,expression1)

#CLOSE=DELAY(CLOSE,1)
expression5 = SecurityLatestValueHolder('closePrice')==SecurityShiftedValueHolder(1,
                    SecurityLatestValueHolder('closePrice'))

#expression5?0:CLOSE-expression4
expression6 =  SecurityIIFValueHolder(expression5,
                                     0, SecurityLatestValueHolder('closePrice') - expression4)

#SUM(expression6,20)
expression7 = SecurityMovingSum(20,expression6)

name = 'alpha59'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = expression7.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

/home/kerry/work/workenv/npyfin/lib/python3.6/site-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in greater


,alpha59,code
trade_date,,
2018-12-28,-1.38,603999
2018-12-28,-0.31,2730
2018-12-28,-1.09,2741
2018-12-28,-0.78,2740
2018-12-28,-0.02,2739


## alpha191-60
$SUM((2*CLOSE-LOW-HIGH)./(HIGH-LOW).*VOLUME,20)$

In [180]:
#(2*CLOSE-LOW-HIGH)
expression1 = 2 * SecurityLatestValueHolder('closePrice') - SecurityLatestValueHolder('lowestPrice') \
            - SecurityLatestValueHolder('highestPrice')
#(HIGH-LOW)
expression2 = (SecurityLatestValueHolder('highestPrice') - SecurityLatestValueHolder('lowestPrice'))

#expression1/expression2*VOLUME
expression3 = expression1 / expression2 * SecurityLatestValueHolder('turnoverVol')

#SUM(expression4,20)
expression4 = SecurityMovingSum(10, expression3)

name = 'alpha60'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = expression4.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

/home/kerry/work/workenv/npyfin/lib/python3.6/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app


,alpha60,code
trade_date,,
2018-12-28,-2.479806e+06,603999
2018-12-28,-6.438088e+06,2730
2018-12-28,1.734773e+06,2741
2018-12-28,-2.019246e+06,2740
2018-12-28,-1.037592e+07,2739


## alpha191-61
$MAX(RANK(DECAYLINEAR(DELTA(VWAP,1),12)),RANK(DECAYLINEAR(RANK(CORR(LOW,MEAN(VOLUME,80),8)),17)))*-1$

In [6]:
#DELTA(VWAP,1)
expression1 = SecurityDeltaValueHolder(1, SecurityLatestValueHolder('vwap'))

#MEAN(VOLUME,80)
expression2 = SecurityMovingAverage(80, 'turnoverVol')

#RANK(DECAYLINEAR(expression1,12))
expression3 = CSPercentileSecurityValueHolder(SecurityXAverageValueHolder(12, expression2))

#RANK(CORR(LOW,expression2,8))
expression4 = CSPercentileSecurityValueHolder(
    SecurityMovingCorrelation(8, SecurityLatestValueHolder('lowestPrice'), expression2))

#RANK(DECAYLINEAR(expression4,17))
expression5 = CSPercentileSecurityValueHolder(SecurityXAverageValueHolder(12, expression2))

#MAX(expression3,expression5)*-1
expression6 = SecurityMaximumValueHolder(expression3, expression5) * -1

name = 'alpha61'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = expression6.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

,alpha61,code
trade_date,,
2018-12-28,-0.630037,603999
2018-12-28,-0.160609,2730
2018-12-28,-0.190194,2741
2018-12-28,-0.342350,2740
2018-12-28,-0.863624,2739


## alpha191-62
$-1*CORR(HIGH,RANK(VOLUME),5)$

In [7]:
#RANK(VOLUME)
expression1 = CSPercentileSecurityValueHolder(SecurityLatestValueHolder('turnoverVol'))
#-1*CORR(HIGH,expression1,5)
expression2 = SecurityMovingCorrelation(5, SecurityLatestValueHolder('highestPrice'), expression1) * -1
name = 'alpha62'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = expression2.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

,alpha62,code
trade_date,,
2018-12-28,-0.469761,603999
2018-12-28,0.890242,2729
2018-12-28,-0.751139,2740
2018-12-28,0.132308,2739
2018-12-28,0.018582,2738


## alpha191-63
$SMA(MAX(CLOSE-DELAY(CLOSE,1),0),6,1)/SMA(ABS(CLOSE-DELAY(CLOSE,1)),6,1)*100$
- MA代替SMA

In [9]:
#CLOSE-DELAY(CLOSE,1)
expression1 = SecurityLatestValueHolder('closePrice') - \
        SecurityShiftedValueHolder(1,SecurityLatestValueHolder('closePrice'))

#MAX(expression1,0)
expression2 = SecurityMaximumValueHolder(expression1, 0)

#ABS(expression1)
expression3 = SecurityAbsValueHolder(expression1)

#SMA(expression2,6,1)/SMA(expression3,6,1)*100
expression4 = SecurityMovingAverage(6,expression2) / SecurityMovingAverage(6,expression3) * 100

name = 'alpha63'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = expression4.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

/home/kerry/work/workenv/npyfin/lib/python3.6/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app


,alpha63,code
trade_date,,
2018-12-28,9.756098,603999
2018-12-28,28.251121,2729
2018-12-28,20.000000,2740
2018-12-28,30.645161,2739
2018-12-28,29.936306,2738


## alpha191-64
$(MAX(RANK(DECAYLINEAR(CORR(RANK(VWAP),RANK(VOLUME),4),4)),RANK(DECAYLINEAR(MAX(CORR(RANK(CLOSE),RANK(MEAN(VOLUME,60)),4),13),14)))*-1)$

In [14]:
#CORR(RANK(VWAP),RANK(VOLUME),4)
expression1 = SecurityMovingCorrelation(4,
                CSPercentileSecurityValueHolder(SecurityLatestValueHolder('vwap')),
                CSPercentileSecurityValueHolder(SecurityLatestValueHolder('turnoverVol')))

#CORR(RANK(CLOSE),RANK(MEAN(VOLUME,60)),4)
expression2 = SecurityMovingCorrelation(4,
            CSPercentileSecurityValueHolder(SecurityLatestValueHolder('closePrice')),
            CSPercentileSecurityValueHolder(SecurityMovingAverage(60, 'turnoverVol')))

#DECAYLINEAR(MAX(expression2,13),14)
expression3 = SecurityXAverageValueHolder(14,SecurityMaximumValueHolder(expression2, 13))

#DECAYLINEAR(expression1,4)
expression4 = SecurityXAverageValueHolder(4,expression1)

#MAX(RANK(expression4),RANK(expression3))*-1
expression5 = SecurityMaximumValueHolder(
                CSPercentileSecurityValueHolder(expression4),
                CSPercentileSecurityValueHolder(expression3)) * -1
name = 'alpha64'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = expression5.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

,alpha64,code
trade_date,,
2018-12-28,-0.500423,603999
2018-12-28,-0.920259,2730
2018-12-28,-0.500423,2741
2018-12-28,-0.748380,2740
2018-12-28,-0.500423,2739


## alpha191-65
$MEAN(CLOSE,6)/CLOSE$

In [10]:
expression1 = SecurityMovingAverage(6, 'closePrice') / SecurityLatestValueHolder('closePrice')
name = 'alpha65'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = expression4.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

/home/kerry/work/workenv/npyfin/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.


,alpha65,code
trade_date,,
2018-12-28,9.756098,603999
2018-12-28,28.251121,2729
2018-12-28,20.000000,2740
2018-12-28,30.645161,2739
2018-12-28,29.936306,2738


## alpha191-66
$(CLOSE-MEAN(CLOSE,6))/MEAN(CLOSE,6)*100$

In [15]:
#(CLOSE-MEAN(CLOSE,6))
expression1 = SecurityLatestValueHolder('closePrice') - SecurityMovingAverage(6, 'closePrice')
#MEAN(CLOSE,6)
expression2 = SecurityMovingAverage(6, 'closePrice')
#expression1/expression2
expression3 = expression1/expression2 * 100

name = 'alpha66'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = expression4.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

,alpha66,code
trade_date,,
2018-12-28,-0.244760,603999
2018-12-28,0.772591,2730
2018-12-28,-0.476461,2741
2018-12-28,0.528373,2740
2018-12-28,-0.210113,2739


## alpha191-67
$SMA(MAX(CLOSE-DELAY(CLOSE,1),0),24,1)/SMA(ABS(CLOSE-DELAY(CLOSE,1)),24,1)*100$
- SMA用MA代替

In [16]:
#SMA(expression2,24,1)/SMA(expression3,24,1)*100


#CLOSE-DELAY(CLOSE,1)
expression1 = SecurityLatestValueHolder('closePrice') - \
        SecurityShiftedValueHolder(1,SecurityLatestValueHolder('closePrice'))

#MAX(expression1,0)
expression2 = SecurityMaximumValueHolder(expression1, 0)

#ABS(expression1)
expression3 = SecurityAbsValueHolder(expression1)

#SMA(expression2,24,1)/SMA(expression3,24,1)*100
expression4 = SecurityMovingAverage(24, expression2) / SecurityMovingAverage(24,expression3) * 100

name = 'alpha67'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = expression4.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

/home/kerry/work/workenv/npyfin/lib/python3.6/site-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide


,alpha66,code
trade_date,,
2018-12-28,46.009390,603999
2018-12-28,45.281124,2729
2018-12-28,38.188976,2740
2018-12-28,51.335878,2739
2018-12-28,39.417178,2738


## alpha191-68
$SMA(((HIGH+LOW)/2-(DELAY(HIGH,1)+DELAY(LOW,1))/2)*(HIGH-LOW)/VOLUME,15,2)$
- 缺少SMA, 用MA代替

In [18]:
#(DELAY(HIGH,1)+DELAY(LOW,1))
expression1 = (SecurityShiftedValueHolder(1,SecurityLatestValueHolder('highestPrice')) + \
                  SecurityShiftedValueHolder(1,SecurityLatestValueHolder('lowestPrice'))) / 2

#((HIGH+LOW)/2-expression1/2)
expression2 = (SecurityLatestValueHolder('highestPrice') + SecurityLatestValueHolder('lowestPrice')) / 2 - \
            expression1 /2

#(HIGH-LOW)/VOLUME
expression3 = (SecurityLatestValueHolder('highestPrice') + SecurityLatestValueHolder('lowestPrice')) / \
                    SecurityLatestValueHolder('turnoverVol')

#SMA(expression2*expression3,15,2)
expression4 = SecurityMovingAverage(15, expression3 * expression2)

name = 'alpha68'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = expression4.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

,alpha66,code
trade_date,,
2018-12-28,0.000007,603999
2018-12-28,0.001040,2729
2018-12-28,0.000025,2740
2018-12-28,0.000043,2739
2018-12-28,0.000283,2738


## alpha191-69
$(SUM(DTM,20)>SUM(DBM,20)?(SUM(DTM,20)-SUM(DBM,20))/SUM(DTM,20):(SUM(DTM,20)=SUM(DBM,20)？0:(SUM(DTM,20)-SUM(DBM,20))/SUM(DBM,20)))$

DTM: $(OPEN<=DELAY(OPEN,1)?0:MAX((HIGH-OPEN),(OPEN-DELAY(OPEN,1))))$

DBM: $(OPEN>=DELAY(OPEN,1)?0:MAX((OPEN-LOW),(OPEN-DELAY(OPEN,1))))$

- expression9 为 空

In [22]:
#(DTM:expression3?0:expression2)

# DBM:(expression5?0:MAX((OPEN-LOW),expression1))

#(OPEN-DELAY(OPEN,1))
expression1 = SecurityLatestValueHolder('openPrice') - SecurityShiftedValueHolder(
        1,SecurityLatestValueHolder('openPrice'))
#MAX((HIGH-OPEN),expression1)
expression2 = SecurityMaximumValueHolder(SecurityLatestValueHolder('highestPrice') - \
                                         SecurityLatestValueHolder('openPrice'), expression1)
#OPEN<=DELAY(OPEN,1)
expression3 = SecurityLatestValueHolder('openPrice') <= SecurityShiftedValueHolder(1,SecurityLatestValueHolder
                                                                                   ('openPrice'))
DTM = SecurityIIFValueHolder(expression3, 0, expression2)


#MAX((OPEN-LOW),expression1)
expression4 = SecurityMaximumValueHolder(SecurityLatestValueHolder('openPrice') - \
                                         SecurityLatestValueHolder('lowestPrice'), expression1)

#OPEN>=DELAY(OPEN,1)
expression5 = SecurityLatestValueHolder('openPrice') >= SecurityShiftedValueHolder(1,SecurityLatestValueHolder
                                                                                   ('openPrice'))

DBM = SecurityIIFValueHolder(expression5, 0, expression4)


#SUM(DBM,20)
expression6 = SecurityMovingSum(6, DBM)
#SUM(DTM,20)
expression7 = SecurityMovingSum(6, DTM)

#expression7-expression6)/expression6
expression8 = (expression7-expression6)/expression6
#(expression7-expression6)/expression7
expression9 = (expression7-expression6)/expression7

#expression7=expression6？0:expression8
expression10 = SecurityIIFValueHolder(expression7==expression6,0,expression8)
#expression7>expression6?expression9:expression10
expression11 = SecurityIIFValueHolder(expression7>expression6,expression9,expression10)


name = 'alpha69'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = expression9.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

/home/kerry/work/workenv/npyfin/lib/python3.6/site-packages/ipykernel_launcher.py:44: RuntimeWarning: invalid value encountered in less_equal
/home/kerry/work/workenv/npyfin/lib/python3.6/site-packages/ipykernel_launcher.py:44: RuntimeWarning: invalid value encountered in greater_equal
/home/kerry/work/workenv/npyfin/lib/python3.6/site-packages/ipykernel_launcher.py:44: RuntimeWarning: invalid value encountered in true_divide


,alpha66,code
trade_date,,


## alpha191-70
$STD(AMOUNT,6)$

In [24]:
expression1 = SecurityMovingStandardDeviation(6,SecurityLatestValueHolder('turnoverValue'))


name = 'alpha70'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = expression1.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

,alpha66,code
trade_date,,
2018-12-28,6.674854e+06,603999
2018-12-28,1.668323e+06,2729
2018-12-28,2.081795e+06,2740
2018-12-28,8.528271e+07,2739
2018-12-28,2.251739e+06,2738


## alpha191-71
$(CLOSE-MEAN(CLOSE,24))/MEAN(CLOSE,24)*100$

In [27]:
#MEAN(CLOSE,24)
expression1 = SecurityMovingAverage(6, 'closePrice')

#(CLOSE-expression1)/expression1*100
expression2 = (SecurityLatestValueHolder('closePrice') - expression1) / expression1 * 100

name = 'alpha71'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = expression2.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

,alpha71,code
trade_date,,
2018-12-28,-2.583026,603999
2018-12-28,-2.241504,2730
2018-12-28,-0.751409,2741
2018-12-28,-3.058387,2740
2018-12-28,-3.350059,2739


## alpha191-72
$SMA((TSMAX(HIGH,6)-CLOSE)/(TSMAX(HIGH,6)-TSMIN(LOW,6))*100,15,1)$
- MA代替SMA

In [28]:
#(TSMAX(HIGH,6)-CLOSE)
expression1 = SecurityMovingMax(6, 'highestPrice') - SecurityLatestValueHolder('closePrice')
#TSMAX(HIGH,6)-TSMIN(LOW,6)
expression2 = SecurityMovingMax(6, 'highestPrice') - SecurityMovingMax(6, 'lowestPrice')
#expression1/expression2*100
expression3 = expression1 / expression2 * 100
# SMA(expression3,15,1)
expression4 = SecurityMovingAverage(15, expression3)


name = 'alpha72'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = expression4.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

/home/kerry/work/workenv/npyfin/lib/python3.6/site-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in true_divide
  del sys.path[0]
/home/kerry/work/workenv/npyfin/lib/python3.6/site-packages/ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in true_divide
  del sys.path[0]


,alpha72,code
trade_date,,
2018-12-28,180.374237,603999
2018-12-28,198.668737,2730
2018-12-28,135.259444,2741
2018-12-28,180.416879,2740
2018-12-28,131.948140,2739


## alpha191-73
$((TSRANK(DECAYLINEAR(DECAYLINEAR(CORR(CLOSE,VOLUME,10),16),4),5)-RANK(DECAYLINEAR(CORR(VWAP,MEAN(VOLUME,30),4),3)))*-1)$
- (expression3-expression5) 相减为null

In [34]:
#DECAYLINEAR(CORR(CLOSE,VOLUME,10),16)
expression1 = SecurityXAverageValueHolder(16,SecurityMovingCorrelation(10, SecurityLatestValueHolder('closePrice'),
                                           SecurityLatestValueHolder('turnoverVol')))
#DECAYLINEAR(expression1,4)
expression2 = SecurityXAverageValueHolder(4, expression1)

#TSRANK(expression2,5)
expression3 = SecurityMovingRank(5, expression2)

#CORR(VWAP,MEAN(VOLUME,30),4)
expression4 = SecurityMovingCorrelation(4, SecurityMovingAverage(30, 'turnoverVol'),
                                       SecurityLatestValueHolder('vwap'))
#RANK(DECAYLINEAR(expression4,3))
expression5 = CSPercentileSecurityValueHolder(SecurityXAverageValueHolder(3, expression4))

#(expression3-expression5)*-1
expression6 = (expression3-expression5)*-1
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = expression3.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

,alpha72,code
trade_date,,
2018-10-08,0.0,1
2018-12-03,0.0,2198
2018-12-03,0.0,2188
2018-12-03,0.0,2189
2018-12-03,0.0,2190


## alpha191-74
$RANK(CORR(SUM(LOW*0.35+VWAP*0.65,20),SUM(MEAN(VOLUME,40),20),7))+RANK(CORR(RANK(VWAP),RANK(VOLUME),6))$

In [35]:
#SUM(LOW*0.35+VWAP*0.65,20)
expression1 = SecurityMovingSum(20, SecurityLatestValueHolder('closePrice') * 0.35 + \
                                SecurityLatestValueHolder('vwap') * 0.65)

#SUM(MEAN(VOLUME,40),20)
expression2 = SecurityMovingSum(20, SecurityMovingAverage(40, 'turnoverVol'))

#RANK(CORR(expression1,expression2,7))
expression3 = CSPercentileSecurityValueHolder(SecurityMovingCorrelation(7, expression1 , expression2))

#CORR(RANK(VWAP),RANK(VOLUME)
expression4 = SecurityMovingCorrelation(6, SecurityLatestValueHolder('vwap'),
                                             SecurityLatestValueHolder('turnoverVol'))
#expression3 + RANK(expression4)
expression5 = expression3 + CSPercentileSecurityValueHolder(expression4)
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = expression5.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

,alpha72,code
trade_date,,
2018-12-28,0.548605,603999
2018-12-28,0.609186,2729
2018-12-28,1.805579,2740
2018-12-28,0.293040,2739
2018-12-28,0.433643,2738


## alpha191-75
$COUNT(CLOSE>OPEN & BANCHMARK_INDEX_CLOSE<BANCHMARK_INDEX_OPEN,50)/COUNT(BANCHMARK_INDEX_CLOSE<BANCHMARK_INDEX_OPEN,50)$
- 缺少BANCHMARK_INDEX_CLOSE

## alpha191-76
$STD(ABS(CLOSE/DELAY(CLOSE,1)-1)/VOLUME,20)/MEAN(ABS(CLOSE/DELAY(CLOSE,1)-1)/VOLUME,20)$

In [36]:

#ABS(CLOSE/DELAY(CLOSE,1) - 1)
expression1 = SecurityAbsValueHolder(SecurityLatestValueHolder('closePrice')/ \
        SecurityShiftedValueHolder(1, SecurityLatestValueHolder('closePrice') - 1))

#STD(expression1/VOLUME,20)
expression2 = SecurityMovingStandardDeviation(20, expression1 / SecurityLatestValueHolder('turnoverVol'))

#MEAN(expression1/VOLUME,20)
expression3 = SecurityMovingAverage(20, expression1 / SecurityLatestValueHolder('turnoverVol'))

## expression2/expression3
expression4 = expression2 / expression3

name = 'alpha76'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = expression4.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

/home/kerry/work/workenv/npyfin/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in true_divide
  app.launch_new_instance()


,alpha76,code
trade_date,,
2018-12-28,0.667353,603999
2018-12-28,0.270617,2729
2018-12-28,0.476097,2740
2018-12-28,0.467233,2739
2018-12-28,0.368332,2738


## alpha191-77
$MIN(RANK(DECAYLINEAR(HIGH*0.5+LOW*0.5-VWAP,20)),RANK(DECAYLINEAR(CORR(HIGH*0.5+LOW*0.5,MEAN(VOLUME,40),3),6)))$

In [37]:
#DECAYLINEAR(HIGH*0.5+LOW*0.5-VWAP,20)
expression1 = SecurityXAverageValueHolder(20, SecurityLatestValueHolder('highestPrice') * 0.5\
        + SecurityLatestValueHolder('lowestPrice') * 0.5 - SecurityLatestValueHolder('vwap'))

#CORR(HIGH*0.5+LOW*0.5,MEAN(VOLUME,40),3)
expression2 = SecurityMovingCorrelation(3, SecurityLatestValueHolder('highestPrice') * 0.5 + \
            SecurityLatestValueHolder('lowestPrice') * 0.5, SecurityMovingAverage(40, 'turnoverVol'))

#DECAYLINEAR(expression2,6)
expression3 = SecurityXAverageValueHolder(6, expression2)

#MIN(RANK(expression1),RANK(expression3))
expression4 = SecurityMinimumValueHolder(CSPercentileSecurityValueHolder(expression1),
                                         CSPercentileSecurityValueHolder(expression3))

name = 'alpha77'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = expression4.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

,alpha77,code
trade_date,,
2018-12-28,0.659059,603999
2018-12-28,0.362919,2730
2018-12-28,0.637081,2741
2018-12-28,0.657368,2740
2018-12-28,0.086221,2739


## alpha191-78
$((HIGH+LOW+CLOSE)/3-MA((HIGH+LOW+CLOSE)/3,12))/(0.015*MEAN(ABS(CLOSE-MEAN((HIGH+LOW+CLOSE)/3,12)),12))$

In [39]:
#(HIGH+LOW+CLOSE)/3
expression1 = (SecurityLatestValueHolder('highestPrice') + SecurityLatestValueHolder('lowestPrice') + \
        SecurityLatestValueHolder('closePrice')) / 3

#MEAN(expression1,12) 或 MA(expression1,12)
expression2 = SecurityMovingAverage(12, expression1)
#ABS(CLOSE-expression2)
expression3 = SecurityAbsValueHolder(SecurityLatestValueHolder('closePrice') - expression2)

#(expression1-expression2)/(0.015*MEAN(expression3,12))
expression4 = (expression1-expression2) / (0.015 * SecurityMovingAverage(12, expression3))

name = 'alpha78'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = expression4.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

/home/kerry/work/workenv/npyfin/lib/python3.6/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app
/home/kerry/work/workenv/npyfin/lib/python3.6/site-packages/ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in true_divide
  from ipykernel import kernelapp as app


,alpha78,code
trade_date,,
2018-12-28,-68.962585,603999
2018-12-28,-90.904540,2729
2018-12-28,-76.144188,2740
2018-12-28,-158.230198,2739
2018-12-28,-77.640729,2738


## alpha191-79
$SMA(MAX(CLOSE-DELAY(CLOSE,1),0),12,1)/SMA(ABS(CLOSE-DELAY(CLOSE,1)),12,1)*100$
- MA代替SMA

In [40]:
#CLOSE-DELAY(CLOSE,1)
expression1 = SecurityLatestValueHolder('closePrice') - \
        SecurityShiftedValueHolder(1,SecurityLatestValueHolder('closePrice'))

#MAX(expression1,0)
expression2 = SecurityMaximumValueHolder(expression1, 0)

#ABS(expression1)
expression3 = SecurityAbsValueHolder(expression1)

#SMA(expression2,12,1)/SMA(expression3,12,1)*100
expression4 = SecurityMovingAverage(12, expression2) / SecurityMovingAverage(12, expression3) * 100

name = 'alpha79'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = expression4.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

/home/kerry/work/workenv/npyfin/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in true_divide
  app.launch_new_instance()


,alpha79,code
trade_date,,
2018-12-28,21.250000,603999
2018-12-28,37.240076,2729
2018-12-28,22.000000,2740
2018-12-28,31.923077,2739
2018-12-28,33.680556,2738


## alpha191-80
$ (VOLUME-DELAY(VOLUME,5))/DELAY(VOLUME,5)*100$

In [42]:
#VOLUME-DELAY(VOLUME,5)
expression1 = SecurityLatestValueHolder('turnoverVol') - \
        SecurityShiftedValueHolder(1,SecurityLatestValueHolder('turnoverVol'))

#expression1/DELAY(VOLUME,5)*100
expression2 = expression1 / SecurityShiftedValueHolder(1,SecurityLatestValueHolder('turnoverVol')) * 100

name = 'alpha80'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = expression2.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

,alpha80,code
trade_date,,
2018-12-28,-35.040453,603999
2018-12-28,29.601453,2729
2018-12-28,-44.467742,2740
2018-12-28,-26.268680,2739
2018-12-28,-5.387791,2738


## alpha191-81
$SMA(VOLUME,21,2)$
- MA代替SMA

In [43]:
expression1 = SecurityMovingAverage(21, SecurityLatestValueHolder('turnoverVol'))

name = 'alpha81'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = expression2.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

,alpha81,code
trade_date,,
2018-12-28,-35.040453,603999
2018-12-28,-34.489613,2730
2018-12-28,-34.132837,2741
2018-12-28,-44.467742,2740
2018-12-28,-26.268680,2739


## alpha191-82
$SMA((TSMAX(HIGH,6)-CLOSE)/(TSMAX(HIGH,6)-TSMIN(LOW,6))*100,20,1)$
- MA代替SMA

In [46]:
#(TSMAX(HIGH,6)-CLOSE)
expression1 = SecurityMovingMax(6, SecurityLatestValueHolder('highestPrice')) - \
        SecurityLatestValueHolder('closePrice')

#(TSMAX(HIGH,6)-TSMIN(LOW,6))
expression2 = SecurityMovingMax(6, SecurityLatestValueHolder('highestPrice')) - \
        SecurityMovingMin(6, SecurityLatestValueHolder('highestPrice'))

#SMA(expression1/expression2*100,20,1)
expression3 = SecurityMovingAverage(20, expression1/expression2*100)

name = 'alpha82'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = expression2.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

,alpha82,code
trade_date,,
2018-12-28,0.33,603999
2018-12-28,0.97,2730
2018-12-28,0.50,2741
2018-12-28,0.39,2740
2018-12-28,1.66,2739


## alpha191-83
$(-1*RANK(COVIANCE(RANK(HIGH),RANK(VOLUME),5)))$
- COVIANCE 缺失

## alpha191-84
$SUM((CLOSE>DELAY(CLOSE,1)?VOLUME:(CLOSE<DELAY(CLOSE,1)?-VOLUME:0)),20)$

In [45]:
#CLOSE>DELAY(CLOSE,1)
expression1 =  SecurityLatestValueHolder('closePrice') > \
            SecurityShiftedValueHolder(1, SecurityLatestValueHolder('closePrice'))
#CLOSE<DELAY(CLOSE,1)
expression2 =  SecurityLatestValueHolder('closePrice') < \
            SecurityShiftedValueHolder(1, SecurityLatestValueHolder('closePrice'))

#(expression2?-VOLUME:0))
expression3 = SecurityIIFValueHolder(expression2, -SecurityLatestValueHolder('turnoverVol'), 
                                     0)
#expression1?VOLUME:expression3
expression4 = SecurityIIFValueHolder(expression1, SecurityLatestValueHolder('turnoverVol'),
                                    expression3)

# SUM(expression4,20)
expression5 = SecurityMovingSum(20, expression4)

name = 'alpha84'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = expression5.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

/home/kerry/work/workenv/npyfin/lib/python3.6/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in greater
/home/kerry/work/workenv/npyfin/lib/python3.6/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in less


,alpha84,code
trade_date,,
2018-12-28,-55914093.0,603999
2018-12-28,-7650516.0,2730
2018-12-28,-6514885.0,2741
2018-12-28,-4652475.0,2740
2018-12-28,69873862.0,2739


## alpha191-85
$TSRANK(VOLUME/MEAN(VOLUME,20),20)*TSRANK(-1*DELTA(CLOSE,7),8)$

In [48]:
#VOLUME/MEAN(VOLUME,20)
expression1 = SecurityLatestValueHolder('turnoverVol') / SecurityMovingAverage(20, 'turnoverVol')
#TSRANK(expression1,20)
expression2 = SecurityMovingRank(20, expression1)
#-1*DELTA(CLOSE,7)
expression3 = SecurityDeltaValueHolder(7, SecurityLatestValueHolder('closePrice')) * -1
#TSRANK(expression3,8)
expression4 = SecurityMovingRank(8, expression3)
#expression2*expression4
expression5 = expression2 * expression4

name = 'alpha85'
df = mkt_df.loc[start_pos:end_pos].reset_index().set_index('trade_date')
result = expression5.transform(df,name=name,category_field='code')
result.sort_values(by='trade_date', ascending=False).head(5)

,alpha85,code
trade_date,,
2018-12-28,0.0,603999
2018-12-28,76.0,2729
2018-12-28,6.0,2740
2018-12-28,66.0,2739
2018-12-28,56.0,2738
